> title : 100_etri_lifelog_preprocess (데이터 전처리 코드)  <br>
 -  코드 실행 전 PATH 변경하세요.
  - PATH  =  '/content/drive/MyDrive/data/ch2025_data_items/share/submissions/

### 🔨 PATH 설정

In [1]:
PATH  =  '/content/drive/MyDrive/data/ch2025_data_items/share/submissions/input' ### <---- 코드 실행 전 PATH 변경하세요.

In [2]:
# 데이터는 구글드라이브에 저장되어 있어서 구글드라이브 마운트를 합니다.
# 데이터 저장 PATH를 변경하시면 아래 구글드라이브 마운트를 주석처리하시면 됩니다.
from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 📚 라이브러리

In [3]:
# 라이브러리 설치
# ! pip install haversine >/dev/null
# ! pip install optuna  >/dev/null
# ! pip install category_encoders >/dev/null
# ! pip install tabpfn  >/dev/null
# ! pip install catboost >/dev/null
# ! pip install torchmetrics >/dev/null

In [4]:
# 기본 라이브러리
import ast
import glob
import os
import random
import re
import sys
from collections import Counter
from datetime import datetime, time, timedelta
from enum import Enum
from functools import reduce
from io import StringIO
from math import asin, cos, radians, sin, sqrt
from pathlib import Path

# 데이터 처리
import numpy as np
import pandas as pd

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 거리 계산
from haversine import haversine  # pip install haversine

# 전처리 및 인코딩
from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 모델링 - 사이킷런
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import (
    KFold,
    StratifiedKFold,
    cross_val_score,
    train_test_split,
)

# 모델링 - LightGBM
import lightgbm as lgb
from lightgbm import LGBMClassifier, early_stopping, log_evaluation

# 모델링 - TabPFN
from tabpfn import TabPFNClassifier

# 모델링 - PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# 기타 유틸
import warnings
from scipy.stats import entropy
from tqdm.auto import tqdm

# 경고 무시
warnings.filterwarnings('ignore')

# pandas 옵션
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%0.4f' % x)

In [5]:
def add_noise(series, noise_level, seed=3):
    rng = np.random.default_rng(seed)
    return series * (1 + noise_level * rng.standard_normal(len(series)))

def calculate_averages(data,name):
    variables = ['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']
    variable_averages = {}
    total_sum = 0
    total_count = 0

    for var in variables:
        values = []
        for entry in data.values():
            if var in entry:  # 키가 존재하는 경우에만 추가
                values.append(entry[var])
        avg = sum(values) / len(values) if values else None  # 누락된 변수 처리
        variable_averages[var] = round(avg, 6) if avg is not None else 'Missing'
        total_sum += sum(values)
        total_count += len(values)

    overall_avg = round(total_sum / total_count, 6) if total_count > 0 else None
    print(f'# 전체 평균 {name}: {overall_avg} {variable_averages}')

    return variable_averages, overall_avg

def remove_highly_correlated_features(X, threshold=0.95):
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    print(f"제거할 feature 개수: {len(to_drop)} / 전체 feature 개수: {X.shape[1]}")
    return X.drop(columns=to_drop), to_drop

def calculate_circular_mean_sleep_time(sleep_times):
    sleep_times = pd.Series(sleep_times).dropna()
    if len(sleep_times) == 0:
        return np.nan  # 혹은 return 0.0 등 기본값 설정 가능

    def hour_to_radian(hour):
        return (hour % 24) / 24 * 2 * np.pi

    radians = np.array([hour_to_radian(t) for t in sleep_times])
    mean_radian = np.arctan2(np.mean(np.sin(radians)), np.mean(np.cos(radians)))
    mean_hour = (mean_radian / (2 * np.pi)) * 24 % 24

    return mean_hour

def circular_mean_sleep_time(times):

    # 결측치 제거
    valid_times = [t for t in times if pd.notna(t)]

    # 유효 데이터 개수 확인
    if len(valid_times) == 0:
        return None  # 결측치만 있는 경우

    # 시간 → 라디안 변환
    radians = [(t % 24) / 24 * 2 * np.pi for t in valid_times]

    # 사인/코사인 평균 계산
    sin_sum = np.mean(np.sin(radians))
    cos_sum = np.mean(np.cos(radians))

    # 평균 각도 계산
    if sin_sum == 0 and cos_sum == 0:
        return np.nan  # 불가능한 경우

    mean_radian = np.arctan2(sin_sum, cos_sum)

    # 평균 시간으로 변환
    mean_hour = (mean_radian / (2 * np.pi)) * 24
    if mean_hour < 0:
        mean_hour += 24

    return f'{int(mean_hour):02d}:{int((mean_hour % 1) * 60):02d}'

def calculate_sleep_duration_min(sleep_time, wake_time):
    """
    취침 시각(sleep_time)과 기상 시각(wake_time)을 입력받아 수면 시간(분) 반환
    단위는 float 시간 (예: 23.5, 6.25)
    """
    if pd.isna(sleep_time) or pd.isna(wake_time):
        return None
    if wake_time < sleep_time:
        wake_time += 24  # 자정 넘긴 경우 보정
    duration = (wake_time - sleep_time) * 60
    return round(duration)

def fill_missing_dates_by_subject(df, date_col='lifelog_date'):

    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    result = []

    for sid, group in df.groupby('subject_id'):
        group = group.sort_values(date_col)

        # 연속 날짜 생성
        full_dates = pd.date_range(start=group[date_col].min(), end=group[date_col].max())
        full_df = pd.DataFrame({date_col: full_dates})
        full_df['subject_id'] = sid

        # 병합
        merged = pd.merge(full_df, group, on=['subject_id', date_col], how='left')

        result.append(merged)

    # 병합 및 정렬
    final_df = pd.concat(result, ignore_index=True).sort_values(['subject_id', date_col])

    return final_df

def get_time_block(hour):
    if 1 <= hour < 5:
        return 'sleeptime'
    else:
        return 'activehour'

class DataType(Enum):
    mACStatus = "mACStatus"
    mActivity = "mActivity"
    mAmbience = "mAmbience"
    mBle = "mBle"
    mGps = "mGps"
    mLight = "mLight"
    mScreenStatus = "mScreenStatus"
    mUsageStats = "mUsageStats"
    mWifi = "mWifi"
    wHr = "wHr"
    wLight = "wLight"
    wPedo = "wPedo"

def load_data(data_type: DataType):
    file_path = f"{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_{data_type.value}.parquet"
    df = pd.read_parquet(file_path)
    df["subject_id"] = df["subject_id"].astype("category")
    df["lifelog_date"] = df["timestamp"].dt.normalize()
    df["month"] = df["timestamp"].dt.month
    df["day"] = df["timestamp"].dt.day
    df["hour"] = df["timestamp"].dt.hour
    df["minute"] = df["timestamp"].dt.minute
    df["weekday"] = df["timestamp"].dt.weekday
    fixed_columns = ["subject_id", "timestamp", "lifelog_date", "month", "day", "hour", "minute", "weekday"]
    columns = df.columns.tolist()
    columns = fixed_columns + [col for col in columns if col not in fixed_columns]
    df = df[columns]
    df = df.sort_values(by=["subject_id", "timestamp"])
    return df

def describe_df(df):
    print(f"# shape:\n{df.shape}\n")
    print(f"# dtypes:\n{df.dtypes}\n")
    # print(f"# head:\n{df.head(3)}\n")
    display(df.head(3))
    nan_stats = df.isna().sum().to_frame(name='missing_count')
    nan_stats['missing_ratio(%)'] = (df.isna().mean() * 100).round(2)
    print(f"# nan_stats:\n" + nan_stats.to_string() + "\n")

def shift_lifelog_date(df, target_hours):
    df = df.copy()
    mask = df["hour"].isin(target_hours) & df["hour"].lt(12)
    df.loc[mask, "lifelog_date"] = df.loc[mask, "lifelog_date"] - pd.Timedelta(days=1)
    df.loc[mask, "day"] = df.loc[mask, "day"] - 1
    df = df.sort_values(by=["subject_id", "lifelog_date", "timestamp"])
    return df

### 📁 데이터 읽기

In [6]:
# 1
mACStatus = pd.read_parquet(f'{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_mACStatus.parquet')
mActivity = pd.read_parquet(f'{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_mActivity.parquet')
mAmbience = pd.read_parquet(f'{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_mAmbience.parquet')
mBle = pd.read_parquet(f'{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_mBle.parquet')
mGps = pd.read_parquet(f'{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_mGps.parquet')
mLight = pd.read_parquet(f'{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_mLight.parquet')
mScreenStatus = pd.read_parquet(f'{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_mScreenStatus.parquet')
mUsageStats = pd.read_parquet(f'{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_mUsageStats.parquet')
mWifi = pd.read_parquet(f'{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_mWifi.parquet')
wHr = pd.read_parquet(f'{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_wHr.parquet')
wLight = pd.read_parquet(f'{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_wLight.parquet')
wPedo = pd.read_parquet(f'{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_wPedo.parquet')

# 2
train = pd.read_csv(f'{PATH}/ETRI_lifelog_dataset/ch2025_metrics_train.csv')
test = pd.read_csv(f'{PATH}/ETRI_lifelog_dataset/ch2025_submission_sample.csv')

# 3
SLEEP_HOURS = tuple(range(0, 5))
MIGHT_GO_TO_SLEEP_HOURS = tuple(range(20, 24)) + tuple(range(0, 2))
MIGHT_WAKEUP_HOURS = tuple(range(6, 10))
ACTIVE_HOURS = tuple(range(7, 24))
WORK_HOURS = tuple(range(7, 19))
FREE_HOURS = tuple(range(19, 24))

# 4 휴일
HOLIDAY_DATES = [
    pd.Timestamp('2024-08-15'),
    pd.Timestamp('2024-09-16'),
    pd.Timestamp('2024-09-17'),
    pd.Timestamp('2024-09-18'),
    pd.Timestamp('2024-10-03'),
    pd.Timestamp('2024-10-09'),
]

### ✔️ mACStatus 핸드폰 충전상태
- Indicates whether the smartphone is currently being charged.
- m_charging : 0/1 상태

In [7]:
def run_length_encoding(arr):
    """Run-Length Encoding"""
    if len(arr) == 0:
        return []

    diffs = np.diff(np.concatenate(([0], arr, [0])))
    run_starts = np.where(diffs == 1)[0]
    run_ends = np.where(diffs == -1)[0]
    return run_ends - run_starts

def process_mACStatus(df):
    status = df["m_charging"].values

    def _process_feature(status):
        if len(status) == 0:
            return 0., 0., 0., 0., 0.

        # charging 상태 비율, 합
        ratio_charging = status.mean()
        sum_charging = status.sum()

        # 상태전이 횟수
        transitions = (status[1:] != status[:-1]).sum()

        lengths = run_length_encoding(status)
        avg_charging_duration = np.mean(lengths) if len(lengths) > 0 else 0
        max_charging_duration = np.max(lengths) if len(lengths) > 0 else 0

        return ratio_charging, sum_charging, transitions, avg_charging_duration, max_charging_duration

    # 하루
    charging_ratio, charging_sum, chargning_transitions, avg_charging_duration, max_charging_duration = _process_feature(status)

    # 잠자는 시간대
    sleep_status = status[df["hour"].isin(SLEEP_HOURS)]
    sleep_charging_ratio, sleep_charging_sum, sleep_charging_transitions, sleep_avg_charging_duration, sleep_max_charging_duration = _process_feature(sleep_status)

    return pd.Series({
        'charging_ratio': charging_ratio,
        'charging_sum': charging_sum,
        'charging_transitions': chargning_transitions,
        'avg_charging_duration': avg_charging_duration,
        'max_charging_duration': max_charging_duration,
        'sleep_charging_ratio': sleep_charging_ratio,
        'sleep_charging_sum': sleep_charging_sum,
        'sleep_charging_transitions': sleep_charging_transitions,
        'sleep_avg_charging_duration': sleep_avg_charging_duration,
        'sleep_max_charging_duration': sleep_max_charging_duration,
    })

mACStatus_ori = load_data(DataType.mACStatus)
mACStatus_ori = shift_lifelog_date(mACStatus_ori, target_hours=SLEEP_HOURS)

mACStatus2  = (
    mACStatus_ori
    .groupby(["subject_id", "lifelog_date"], group_keys=False, as_index=False, sort=False, observed=True)
    .apply(process_mACStatus)
    .reset_index(drop=True)
)

describe_df(mACStatus2)

# shape:
(803, 12)

# dtypes:
subject_id                           category
lifelog_date                   datetime64[ns]
charging_ratio                        float64
charging_sum                          float64
charging_transitions                  float64
avg_charging_duration                 float64
max_charging_duration                 float64
sleep_charging_ratio                  float64
sleep_charging_sum                    float64
sleep_charging_transitions            float64
sleep_avg_charging_duration           float64
sleep_max_charging_duration           float64
dtype: object



,subject_id,lifelog_date,charging_ratio,charging_sum,charging_transitions,avg_charging_duration,max_charging_duration,sleep_charging_ratio,sleep_charging_sum,sleep_charging_transitions,sleep_avg_charging_duration,sleep_max_charging_duration
0,id01,2024-06-26,0.1498,147.0000,22.0000,13.3636,41.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,id01,2024-06-27,0.1650,231.0000,33.0000,13.5882,65.0000,0.0300,9.0000,1.0000,9.0000,9.0000
2,id01,2024-06-28,0.3764,527.0000,28.0000,35.1333,356.0000,1.0000,280.0000,0.0000,280.0000,280.0000


# nan_stats:
                             missing_count  missing_ratio(%)
subject_id                               0            0.0000
lifelog_date                             0            0.0000
charging_ratio                           0            0.0000
charging_sum                             0            0.0000
charging_transitions                     0            0.0000
avg_charging_duration                    0            0.0000
max_charging_duration                    0            0.0000
sleep_charging_ratio                     0            0.0000
sleep_charging_sum                       0            0.0000
sleep_charging_transitions               0            0.0000
sleep_avg_charging_duration              0            0.0000
sleep_max_charging_duration              0            0.0000



### ✔️ mActivity 추정행동
- Value calculated by the Google Activity Recognition API.
 - 0 : IN_VEHICLE
 - 1 : ON_BICYCLE
 - 2 : ON_FOOT
 - 3 : STILL (not moving)
 - 4 : UNKNOWN
 - 5 : TILTING (This often occurs when a device is picked up from a desk or a user who is sitting stands up.)
 - 7 : WALKING
 - 8 : RUNNING
- 근무시간   : 오전 7시부터 오후 6시까지
- 근무외시간 : 오후6시부터 12시까지

In [8]:
def process_mActivity(df):
    activity = df["m_activity"].values.astype("int8")

    EXCLUDE_ACTIVITY = [3, 4]
    WALKING_ACTIVITY = [1, 2, 7, 8]
    VEHICLE_ACTIVITY = [0]

    def _process_feature(activity):
        if len(activity) == 0:
            return 0., 0., 0.

        # Walking minutes
        walking_minutes = np.isin(activity, WALKING_ACTIVITY).sum()

        # Vehicle minutes
        vehicle_minutes = np.isin(activity, VEHICLE_ACTIVITY).sum()

        # Activity minutes
        activity_minutes = (1 - np.isin(activity, EXCLUDE_ACTIVITY)).sum()

        return walking_minutes, vehicle_minutes, activity_minutes

    # 하루
    walking_minutes, vehicle_minutes, activity_minutes = _process_feature(activity)

    # 잠자는 시간대
    sleep_walking_minutes, sleep_vehicle_minutes, sleep_activity_minutes = _process_feature(activity[df["hour"].isin(SLEEP_HOURS)])

    return pd.Series({
        'walking_minutes': walking_minutes,
        'vehicle_minutes': vehicle_minutes,
        'activity_minutes': activity_minutes,
        'sleep_walking_minutes': sleep_walking_minutes,
        'sleep_vehicle_minutes': sleep_vehicle_minutes,
        'sleep_activity_minutes': sleep_activity_minutes,
    })

mActivity_ori = load_data(DataType.mActivity)
mActivity_ori = shift_lifelog_date(mActivity_ori, target_hours=SLEEP_HOURS)

mActivity21 = (
    mActivity_ori
    .groupby(["subject_id", "lifelog_date"], group_keys=False, as_index=False, sort=False, observed=True)
    .apply(process_mActivity)
    .reset_index(drop=True)
)

describe_df(mActivity21)

# shape:
(803, 8)

# dtypes:
subject_id                      category
lifelog_date              datetime64[ns]
walking_minutes                  float64
vehicle_minutes                  float64
activity_minutes                 float64
sleep_walking_minutes            float64
sleep_vehicle_minutes            float64
sleep_activity_minutes           float64
dtype: object



,subject_id,lifelog_date,walking_minutes,vehicle_minutes,activity_minutes,sleep_walking_minutes,sleep_vehicle_minutes,sleep_activity_minutes
0,id01,2024-06-26,32.0000,89.0000,121.0000,0.0000,0.0000,0.0000
1,id01,2024-06-27,31.0000,211.0000,242.0000,0.0000,0.0000,0.0000
2,id01,2024-06-28,37.0000,161.0000,198.0000,0.0000,0.0000,0.0000


# nan_stats:
                        missing_count  missing_ratio(%)
subject_id                          0            0.0000
lifelog_date                        0            0.0000
walking_minutes                     0            0.0000
vehicle_minutes                     0            0.0000
activity_minutes                    0            0.0000
sleep_walking_minutes               0            0.0000
sleep_vehicle_minutes               0            0.0000
sleep_activity_minutes              0            0.0000



In [9]:
mActivity = pd.read_parquet(f'{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_mActivity.parquet')
mActivity['lifelog_date'] = mActivity['timestamp'].astype(str).str[:10]

In [10]:
# 활동 데이터 원-핫 인코딩
"""활동 코드(m_activity)를 원-핫 인코딩하여 각 활동 유형별 컬럼 생성"""

mActivity = pd.merge(
    mActivity,
    pd.get_dummies(mActivity, columns=["m_activity"], prefix="m_activity", dtype=int),
    how="left",
    on=["subject_id", "timestamp","lifelog_date"],
)

In [11]:
# 데이터 집계 함수 정의
def fn_love_aespa(
    df_input: pd.DataFrame, # 입력 데이터프레임
    str_value_col: str, # 집계할 컬럼명
    str_agg_func: str = "mean", # 집계 함수 (mean, median, mode, min, max, std, sum)
    str_freq: str = "30min", # 시간 간격 (30min, 60min, 120min, 240min, 360min 등)
) -> pd.DataFrame:
    # 데이터프레임 복사 및 timestamp 열을 datetime 형식으로 변환
    df_input_copy = df_input.copy()
    df_input_copy["timestamp"] = pd.to_datetime(df_input_copy["timestamp"])

    # 집계 결과 컬럼명 생성: @컬럼명@시간간격@집계함수
    str_agg_col_name = f"@{str_value_col}@{str_freq}@{str_agg_func}"

    # 집계 함수 설정 (mode는 별도 처리 필요)
    dict_aggregation = {}
    if str_agg_func == "mode":
        mode_agg_func = lambda x: (x.mode().iloc[0] if not x.mode().empty else np.nan)
        dict_aggregation[str_agg_col_name] = (str_value_col, mode_agg_func)
    else:
        dict_aggregation[str_agg_col_name] = (str_value_col, str_agg_func)

    # 그룹별 데이터 집계 수행
    df_agg = (
        df_input_copy.groupby(["subject_id", pd.Grouper(key="timestamp", freq=str_freq)]).agg(**dict_aggregation).reset_index()
    )

    # 날짜 및 시간 정보 추출
    df_agg["lifelog_date"] = df_agg["timestamp"].dt.date.astype(str)
    df_agg["hh24mi"] = df_agg["timestamp"].dt.strftime("%Hh%Mm")

    # 피벗 테이블로 데이터 재구성 (subject_id, lifelog_date 기준으로 시간대별 값 배치)
    df_pivot = df_agg.pivot_table(
        index=["subject_id", "lifelog_date"],
        columns="hh24mi",
        values=str_agg_col_name,
    )

    # 컬럼 이름 재구성 및 인덱스 초기화
    list_hh23mi_col = list(df_pivot.columns)
    df_pivot = df_pivot.reindex(columns=list_hh23mi_col).reset_index()
    list_hour_col = {hh24mi: f"{str_value_col}@{str_freq}@{str_agg_func}@{hh24mi}" for hh24mi in list_hh23mi_col}
    df_pivot = df_pivot.rename(columns=list_hour_col)

    return df_pivot

In [12]:
# MET 값 매핑
"""
각 활동 코드에 해당하는 MET(Metabolic Equivalent of Task) 값 할당
MET는 신체 활동의 에너지 소비량을 측정하는 단위

활동 코드별 MET 값:
    0: 1.3 MET (가벼운 좌식 활동)
    1: 8.0 MET (격렬한 활동)
    3: 1.2 MET (매우 가벼운 활동)
    4: 3.0 MET (중간 강도 활동)
    7: 3.5 MET (중간 강도 활동)
    8: 10.0 MET (매우 격렬한 활동)
"""

dict_met_value = {0: 1.3, 1: 8.0, 3: 1.2, 4: 3.0, 7: 3.5, 8: 10.0}
for activity, met in dict_met_value.items():
    mActivity.loc[mActivity["m_activity"].isin([activity]), "m_activity_met"] = met

mActivity.head(5)

,subject_id,timestamp,m_activity,lifelog_date,m_activity_0,m_activity_1,m_activity_3,m_activity_4,m_activity_7,m_activity_8,m_activity_met
0,id01,2024-06-26 12:03:00,4,2024-06-26,0,0,0,1,0,0,3.0000
1,id01,2024-06-26 12:04:00,0,2024-06-26,1,0,0,0,0,0,1.3000
2,id01,2024-06-26 12:05:00,0,2024-06-26,1,0,0,0,0,0,1.3000
3,id01,2024-06-26 12:06:00,0,2024-06-26,1,0,0,0,0,0,1.3000
4,id01,2024-06-26 12:07:00,0,2024-06-26,1,0,0,0,0,0,1.3000


In [13]:
# 활동 데이터 집계
df_agg_activity_std = fn_love_aespa(df_input=mActivity,
                                    str_value_col="m_activity",
                                    # "mean", "median", "mode", "min", "max", "std"
                                    str_agg_func="std",
                                    # "30min", "60min", "120min", "240min", "360min", "480min", "720min", "1440min"
                                    str_freq="240min",
                                    )

df_agg_activity_met_std = fn_love_aespa(df_input=mActivity,
                                    str_value_col="m_activity_met",
                                    # "mean", "median", "mode", "sum", "min", "max", "std"
                                    str_agg_func="std",
                                    # "30min", "60min", "120min", "240min", "360min", "480min", "720min", "1440min"
                                    str_freq="240min",
                                    )

df_agg_activity_met_sum = fn_love_aespa(df_input=mActivity,
                                    str_value_col="m_activity_met",
                                    # "mean", "median", "mode", "sum", "min", "max", "std"
                                    str_agg_func="sum",
                                    # "30min", "60min", "120min", "240min", "360min", "480min", "720min", "1440min"
                                    str_freq="240min",
                                    )

df_agg_activity_0_std = fn_love_aespa(df_input=mActivity,
                                    str_value_col="m_activity_0",
                                    # "mean", "median", "mode", "sum", "min", "max", "std"
                                    str_agg_func="std",
                                    # "30min", "60min", "120min", "240min", "360min", "480min", "720min", "1440min"
                                    str_freq="240min",
                                    )

df_agg_activity_0_sum = fn_love_aespa(df_input=mActivity,
                                    str_value_col="m_activity_0",
                                    # "mean", "median", "mode", "sum", "min", "max", "std"
                                    str_agg_func="sum",
                                    # "30min", "60min", "120min", "240min", "360min", "480min", "720min", "1440min"
                                    str_freq="240min",
                                    )

In [14]:
# 병합 기준 key
merge_keys = ['subject_id', 'lifelog_date']

# 세 개 데이터프레임 순차 병합
mActivity22 = (
    df_agg_activity_std
    .merge(df_agg_activity_met_std, on=merge_keys, how='outer')
    .merge(df_agg_activity_met_sum, on=merge_keys, how='outer')
    .merge(df_agg_activity_0_std, on=merge_keys, how='outer')
    .merge(df_agg_activity_0_sum, on=merge_keys, how='outer')
)

# check
print(mActivity22.shape)

(700, 32)


### ✔️ mAmbience 추정주변소리
- Ambient sound identification labels and their respective probabilities.

In [15]:
def process_mAmbience(df):
    ambience = df["m_ambience"].values  # [[label, prob], ...], [[label, prob], ...]

    def _process_feature(ambience):
        labels = set()

        for amb in ambience:
            labels_, _ = zip(*amb)
            labels.update(labels_)

        unique_label_count = len(labels)
        snor_count = len(list(filter(lambda x: "snor" in x.lower(), labels)))

        return unique_label_count, snor_count

    # 활동시간
    active_hour_unique_label_count, active_hour_snor_count = _process_feature(ambience[df["hour"].isin(ACTIVE_HOURS)])

    # 잠자는시간
    sleep_hour_unique_label_count, sleep_hour_snor_count = _process_feature(ambience[df["hour"].isin(SLEEP_HOURS)])

    return pd.Series({
        'active_hour_unique_label_count': active_hour_unique_label_count,
        'active_hour_snor_count': active_hour_snor_count,
        'sleep_hour_unique_label_count': sleep_hour_unique_label_count,
        'sleep_hour_snor_count': sleep_hour_snor_count,
    })

mAmbience_ori = load_data(DataType.mAmbience)
mAmbience_ori = shift_lifelog_date(mAmbience_ori, target_hours=SLEEP_HOURS)

mAmbience2 = (
    mAmbience_ori
    .groupby(["subject_id", "lifelog_date"], group_keys=False, as_index=False, sort=False, observed=True)
    .apply(process_mAmbience)
    .reset_index(drop=True)
)

describe_df(mAmbience2)

# shape:
(803, 6)

# dtypes:
subject_id                              category
lifelog_date                      datetime64[ns]
active_hour_unique_label_count             int64
active_hour_snor_count                     int64
sleep_hour_unique_label_count              int64
sleep_hour_snor_count                      int64
dtype: object



,subject_id,lifelog_date,active_hour_unique_label_count,active_hour_snor_count,sleep_hour_unique_label_count,sleep_hour_snor_count
0,id01,2024-06-26,265,2,10,0
1,id01,2024-06-27,10,0,10,0
2,id01,2024-06-28,14,0,10,0


# nan_stats:
                                missing_count  missing_ratio(%)
subject_id                                  0            0.0000
lifelog_date                                0            0.0000
active_hour_unique_label_count              0            0.0000
active_hour_snor_count                      0            0.0000
sleep_hour_unique_label_count               0            0.0000
sleep_hour_snor_count                       0            0.0000



### ✔️ mBle 블루투스
- Bluetooth devices around individual subject.
 - 7936 : Wearable, Headset, AV Device
 - 1796 : Peripheral (입력장치) 계열
 - 0 : 정보 없음 또는 알 수 없음(Unknown)
 - 1084 : Audio/Video (스피커, 헤드셋, 이어폰, TV 등)
 - 524 : Phone (휴대폰, 스마트폰)
 - 1060 : Headphones
 - 284 : commputer (PC, 노트북, PDA)

In [16]:
def process_mBle(df):
    ble = df["m_ble"].values  # [[{"address": "xx:xx:xx:xx:xx:xx", "device_class": "0", "rssi": -70}, ...], [...], ...]

    def _process_feature(ble):
        if len(ble) == 0:
            return 0., 0., 0., 0., 0.

        rssi = []
        devices = []
        for ble_data in ble:
            for device in ble_data:
                rssi.append(device["rssi"])
                devices.append(device["device_class"])

        rssi = np.array(rssi)
        rssi_mean = rssi.mean() if len(rssi) > 0 else 0
        rssi_min = rssi.min() if len(rssi) > 0 else 0
        rssi_max = rssi.max() if len(rssi) > 0 else 0

        unknown_count = devices.count("0")
        others_count = len(devices) - unknown_count
        others_ratio = others_count / len(devices) if len(devices) > 0 else 0
        unknown_ratio = unknown_count / len(devices) if len(devices) > 0 else 0

        return rssi_mean, rssi_min, rssi_max, others_ratio, unknown_ratio

    # 일할때
    work_hour_rssi_mean, work_hour_rssi_min, work_hour_rssi_max, work_hour_others_ratio, work_hour_unknown_ratio = _process_feature(ble[df["hour"].isin(WORK_HOURS)])

    # 퇴근후
    free_hour_rssi_mean, free_hour_rssi_min, free_hour_rssi_max, free_hour_others_ratio, free_hour_unknown_ratio = _process_feature(ble[df["hour"].isin(FREE_HOURS)])

    # 잠자는시간
    sleep_hour_rssi_mean, sleep_hour_rssi_min, sleep_hour_rssi_max, sleep_hour_others_ratio, sleep_hour_unknown_ratio = _process_feature(ble[df["hour"].isin(SLEEP_HOURS)])

    return pd.Series({
        'work_hour_rssi_mean': work_hour_rssi_mean,
        'work_hour_rssi_min': work_hour_rssi_min,
        'work_hour_rssi_max': work_hour_rssi_max,
        'work_hour_others_ratio': work_hour_others_ratio,
        'work_hour_unknown_ratio': work_hour_unknown_ratio,
        'free_hour_rssi_mean': free_hour_rssi_mean,
        'free_hour_rssi_min': free_hour_rssi_min,
        'free_hour_rssi_max': free_hour_rssi_max,
        'free_hour_others_ratio': free_hour_others_ratio,
        'free_hour_unknown_ratio': free_hour_unknown_ratio,
        'sleep_hour_rssi_mean': sleep_hour_rssi_mean,
        'sleep_hour_rssi_min': sleep_hour_rssi_min,
        'sleep_hour_rssi_max': sleep_hour_rssi_max,
        'sleep_hour_others_ratio': sleep_hour_others_ratio,
        'sleep_hour_unknown_ratio': sleep_hour_unknown_ratio
    })

mBle_ori = load_data(DataType.mBle)
mBle_ori = shift_lifelog_date(mBle_ori, target_hours=SLEEP_HOURS)

mBle2 = (
    mBle_ori
    .groupby(["subject_id", "lifelog_date"], group_keys=False, as_index=False, sort=False, observed=True)
    .apply(process_mBle)
    .reset_index(drop=True)
)

describe_df(mBle2)

# shape:
(709, 17)

# dtypes:
subject_id                        category
lifelog_date                datetime64[ns]
work_hour_rssi_mean                float64
work_hour_rssi_min                 float64
work_hour_rssi_max                 float64
work_hour_others_ratio             float64
work_hour_unknown_ratio            float64
free_hour_rssi_mean                float64
free_hour_rssi_min                 float64
free_hour_rssi_max                 float64
free_hour_others_ratio             float64
free_hour_unknown_ratio            float64
sleep_hour_rssi_mean               float64
sleep_hour_rssi_min                float64
sleep_hour_rssi_max                float64
sleep_hour_others_ratio            float64
sleep_hour_unknown_ratio           float64
dtype: object



,subject_id,lifelog_date,work_hour_rssi_mean,work_hour_rssi_min,work_hour_rssi_max,work_hour_others_ratio,work_hour_unknown_ratio,free_hour_rssi_mean,free_hour_rssi_min,free_hour_rssi_max,free_hour_others_ratio,free_hour_unknown_ratio,sleep_hour_rssi_mean,sleep_hour_rssi_min,sleep_hour_rssi_max,sleep_hour_others_ratio,sleep_hour_unknown_ratio
0,id01,2024-06-26,-74.0904,-94.0000,-27.0000,0.0590,0.9410,-77.2213,-92.0000,-43.0000,0.0791,0.9209,0.0000,0.0000,0.0000,0.0000,0.0000
1,id01,2024-06-27,-73.7473,-94.0000,-34.0000,0.0614,0.9386,-74.6667,-91.0000,-42.0000,0.1167,0.8833,0.0000,0.0000,0.0000,0.0000,0.0000
2,id01,2024-06-28,-75.7993,-92.0000,-39.0000,0.0467,0.9533,-77.2558,-94.0000,-51.0000,0.3256,0.6744,0.0000,0.0000,0.0000,0.0000,0.0000


# nan_stats:
                          missing_count  missing_ratio(%)
subject_id                            0            0.0000
lifelog_date                          0            0.0000
work_hour_rssi_mean                   0            0.0000
work_hour_rssi_min                    0            0.0000
work_hour_rssi_max                    0            0.0000
work_hour_others_ratio                0            0.0000
work_hour_unknown_ratio               0            0.0000
free_hour_rssi_mean                   0            0.0000
free_hour_rssi_min                    0            0.0000
free_hour_rssi_max                    0            0.0000
free_hour_others_ratio                0            0.0000
free_hour_unknown_ratio               0            0.0000
sleep_hour_rssi_mean                  0            0.0000
sleep_hour_rssi_min                   0            0.0000
sleep_hour_rssi_max                   0            0.0000
sleep_hour_others_ratio               0            0.0000
s

### ✔️ mGps, GPS 기반 핸드폰 위치
- Multiple GPS coordinates measured within a single minute using the smartphone.
- speed가 1보다 큰경우 정지 상태가 아니고 움직이고 있다고 판단
 - 0.5-2 : 걸어서 이동하는 경우  
 - 2-5 : 조깅
 - 5 이상 : 차를 타고 이동하는 경우

In [17]:
from datetime import datetime

def haversine_np(lon1, lat1, lon2, lat2, radius=6371):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return radius * c

def process_mGps(df):
    gps = df["m_gps"].values  # [[{'altitude': 110.6, 'latitude': 0.2077385, 'longitude': 0.170027, 'speed': 0.0}, ...], ...]
    timestamps = df["timestamp"].values

    def _process_feature(gps, timestamps):
        if len(gps) == 0:
            return 0., 0., 0., 0., 0., 0., 0., np.array([])

        # n-분 단위
        latitudes = []
        longitudes = []
        altitudes = []
        speeds = []
        minutes = []  # 누적 분

        for i, (gps_data, timestamp) in enumerate(zip(gps, timestamps)):
            _latitudes = []
            _longitudes = []
            _altitudes = []
            _speeds = []
            for data in gps_data:
                _latitudes.append(data["latitude"])
                _longitudes.append(data["longitude"])
                _altitudes.append(data["altitude"])
                _speeds.append(data["speed"])

            latitudes.append(np.mean(_latitudes))
            longitudes.append(np.mean(_longitudes))
            altitudes.append(np.mean(_altitudes))
            speeds.append(np.mean(_speeds))
            minutes.append(1 if i == 0 else pd.Timedelta(timestamps[i] - timestamps[i-1]).total_seconds() / 60)

        latitudes = np.array(latitudes)
        longitudes = np.array(longitudes)
        altitudes = np.array(altitudes)
        speeds = np.array(speeds)
        minutes = np.array(minutes)

        walk_minutes = minutes[(speeds >= 0.5) & (speeds < 2.0)].sum()
        jog_minutes = minutes[(2.0 <= speeds) & (speeds < 5.0)].sum()
        vehicle_minutes = minutes[(5.0 <= speeds)].sum()

        # 속도
        mean_speed = speeds.mean() if len(speeds) > 0 else 0
        max_speed = speeds.max() if len(speeds) > 0 else 0
        min_speed = speeds.min() if len(speeds) > 0 else 0

        # 이동거리
        distance = haversine_np(longitudes[:-1], latitudes[:-1], longitudes[1:], latitudes[1:]).sum()

        return walk_minutes, jog_minutes, vehicle_minutes, mean_speed, max_speed, min_speed, distance, speeds

    # 하루
    active_hour_walk_minutes, active_hour_jog_minutes, active_hour_vehicle_minutes, active_hour_mean_speed, active_hour_max_speed, active_hour_min_speed, active_hour_distance, _ = _process_feature(gps[df["hour"].isin(ACTIVE_HOURS)], timestamps[df["hour"].isin(ACTIVE_HOURS)])

    # 잠자는 시간대
    sleep_hour_walk_minutes, sleep_hour_jog_minutes, sleep_hour_vehicle_minutes, sleep_hour_mean_speed, sleep_hour_max_speed, sleep_hour_min_speed, sleep_hour_distance, _ = _process_feature(gps[df["hour"].isin(SLEEP_HOURS)], timestamps[df["hour"].isin(SLEEP_HOURS)])

    # 일어날 때
    _, _, _, _, _, _, _, might_wakeup_speeds = _process_feature(gps[df["hour"].isin(MIGHT_WAKEUP_HOURS)], timestamps[df["hour"].isin(MIGHT_WAKEUP_HOURS)])
    might_wakeup_timestamps = timestamps[df["hour"].isin(MIGHT_WAKEUP_HOURS)]
    wakeup_timestamps = might_wakeup_timestamps[(might_wakeup_speeds > 1.0)]
    first_move_datetime = (
        pd.to_datetime(wakeup_timestamps[0]) if len(wakeup_timestamps) > 0
        else pd.to_datetime(might_wakeup_timestamps[-1]) if len(might_wakeup_timestamps) > 0
        else pd.to_datetime(datetime(2024, 1, 1, MIGHT_WAKEUP_HOURS[-1], 0, 0))  # default to the last hour of the range
    )
    first_wakeup_minutes = (first_move_datetime.hour if first_move_datetime.hour > 12 else first_move_datetime.hour + 24) * 60 + first_move_datetime.minute

    return pd.Series({
        'active_hour_walk_minutes': active_hour_walk_minutes,
        'active_hour_jog_minutes': active_hour_jog_minutes,
        'active_hour_vehicle_minutes': active_hour_vehicle_minutes,
        'active_hour_mean_speed': active_hour_mean_speed,
        'active_hour_max_speed': active_hour_max_speed,
        'active_hour_min_speed': active_hour_min_speed,
        'active_hour_distance': active_hour_distance,
        'exercise_flag': 1 if active_hour_jog_minutes > 10 else 0,  # n분 이상 조깅한 경우
        'sleep_hour_walk_minutes': sleep_hour_walk_minutes,
        'sleep_hour_jog_minutes': sleep_hour_jog_minutes,
        'sleep_hour_vehicle_minutes': sleep_hour_vehicle_minutes,
        'sleep_hour_mean_speed': sleep_hour_mean_speed,
        'sleep_hour_max_speed': sleep_hour_max_speed,
        'sleep_hour_min_speed': sleep_hour_min_speed,
        'sleep_hour_distance': sleep_hour_distance,
        "mgps_first_wakeup_minutes": first_wakeup_minutes,
    })


mGps_ori = load_data(DataType.mGps)
mGps_ori = shift_lifelog_date(mGps_ori, target_hours=SLEEP_HOURS)

mGps2 = (
    mGps_ori
    .groupby(["subject_id", "lifelog_date"], group_keys=False, as_index=False, sort=False, observed=True)
    .apply(process_mGps)
    .reset_index(drop=True)
)

describe_df(mGps2)

# shape:
(759, 18)

# dtypes:
subject_id                           category
lifelog_date                   datetime64[ns]
active_hour_walk_minutes              float64
active_hour_jog_minutes               float64
active_hour_vehicle_minutes           float64
active_hour_mean_speed                float64
active_hour_max_speed                 float64
active_hour_min_speed                 float64
active_hour_distance                  float64
exercise_flag                         float64
sleep_hour_walk_minutes               float64
sleep_hour_jog_minutes                float64
sleep_hour_vehicle_minutes            float64
sleep_hour_mean_speed                 float64
sleep_hour_max_speed                  float64
sleep_hour_min_speed                  float64
sleep_hour_distance                   float64
mgps_first_wakeup_minutes             float64
dtype: object



,subject_id,lifelog_date,active_hour_walk_minutes,active_hour_jog_minutes,active_hour_vehicle_minutes,active_hour_mean_speed,active_hour_max_speed,active_hour_min_speed,active_hour_distance,exercise_flag,sleep_hour_walk_minutes,sleep_hour_jog_minutes,sleep_hour_vehicle_minutes,sleep_hour_mean_speed,sleep_hour_max_speed,sleep_hour_min_speed,sleep_hour_distance,mgps_first_wakeup_minutes
0,id01,2024-06-26,68.0000,32.0000,19.0000,0.5775,19.0505,0.0000,16.7900,1.0000,36.0000,0.0000,0.0000,0.1812,1.6664,0.0000,0.1752,1980.0000
1,id01,2024-06-27,136.0000,61.0000,66.0000,1.0368,24.2032,0.0000,32.2769,1.0000,1.0000,0.0000,0.0000,0.0424,0.5080,0.0000,0.0647,1831.0000
2,id01,2024-06-28,106.0000,68.0000,42.0000,0.8060,24.1712,0.0001,35.5108,1.0000,19.0000,0.0000,0.0000,0.3053,0.8415,0.0987,0.4590,1834.0000


# nan_stats:
                             missing_count  missing_ratio(%)
subject_id                               0            0.0000
lifelog_date                             0            0.0000
active_hour_walk_minutes                 0            0.0000
active_hour_jog_minutes                  0            0.0000
active_hour_vehicle_minutes              0            0.0000
active_hour_mean_speed                   0            0.0000
active_hour_max_speed                    0            0.0000
active_hour_min_speed                    0            0.0000
active_hour_distance                     0            0.0000
exercise_flag                            0            0.0000
sleep_hour_walk_minutes                  0            0.0000
sleep_hour_jog_minutes                   0            0.0000
sleep_hour_vehicle_minutes               0            0.0000
sleep_hour_mean_speed                    0            0.0000
sleep_hour_max_speed                     0            0.0000
sleep_hour_

### ✔️ mLight 주변 밝기
- Ambient light measured by the smartphone.
 - 어두운 밤	0.1 ~ 1 lux	캄캄한 방, 달빛 없는 밤
 - 가로등 켜진 거리	10 ~ 20 lux	흐릿한 외부 조명
 - 실내 조명	100 ~ 500 lux	사무실, 일반 거실
 - 밝은 실외	10,000 ~ 25,000 lux	맑은 날 햇빛
 - 직사광선 아래	30,000 ~ 100,000 lux	여름 한낮, 매우 강한 햇빛

In [18]:
mLight['lifelog_date'] = mLight['timestamp'].astype(str).str[:10]
# mLight = fill_missing_dates_by_subject(mLight)
mLight.head()

,subject_id,timestamp,m_light,lifelog_date
0,id01,2024-06-26 12:03:00,534.0000,2024-06-26
1,id01,2024-06-26 12:13:00,846.0000,2024-06-26
2,id01,2024-06-26 12:23:00,826.0000,2024-06-26
3,id01,2024-06-26 12:33:00,851.0000,2024-06-26
4,id01,2024-06-26 12:43:00,428.0000,2024-06-26


In [19]:
def process_mLight(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour
    df['is_night'] = df['hour'].apply(lambda h: h >= 22 or h < 6)

    # 하루 요약 통계
    daily_light = df.groupby(['subject_id', 'lifelog_date']).agg(
        light_mean=('m_light', 'mean'),
        light_std=('m_light', 'std'),
        light_max=('m_light', 'max'),
        light_min=('m_light', 'min'),
        light_night_mean=('m_light', lambda x: x[df.loc[x.index, 'is_night']].mean()),
        light_day_mean=('m_light', lambda x: x[~df.loc[x.index, 'is_night']].mean()),
        light_night_ratio=('is_night', 'mean')
    ).reset_index()

    results = []

    for subject_id, group in tqdm(df.groupby('subject_id'), desc="Processing light-based sleep detection"):
        group = group.sort_values('timestamp').reset_index(drop=True)

        recorded_dates = set()
        sleeping = False
        zero_count = 0
        first_zero_time = None

        for i in range(len(group)):
            light = group.loc[i, 'm_light']
            hour = group.loc[i, 'hour']

            if light == 0:
                zero_count += 1
                if zero_count == 1:
                    first_zero_time = group.loc[i, 'timestamp']
                if zero_count >= 6 and not sleeping:
                    sleep_hour = first_zero_time.hour
                    if (sleep_hour >= 21 or sleep_hour <= 2):
                        sleeping = True
            else:
                if sleeping:
                    candidate_wakeup = group.loc[i, 'timestamp']
                    wake_hour = candidate_wakeup.hour

                    if 5 <= wake_hour <= 9 and first_zero_time is not None:
                        wake_time = candidate_wakeup
                        sleep_time = first_zero_time
                        duration_min = (wake_time - sleep_time).total_seconds() / 60

                        if 0 < duration_min <= 840:
                            sleep_duration = duration_min
                        else:
                            sleep_duration = np.nan

                        lifelog_date = wake_time.date() + pd.Timedelta(days=-1)

                        if lifelog_date not in recorded_dates:
                            results.append({
                                'subject_id': subject_id,
                                'lifelog_date': lifelog_date,
                                'sleep_duration_min_mLight': sleep_duration,
                                'sleep_time_min_mLight': sleep_time.hour * 60 + sleep_time.minute,
                                'wake_time_min_mLight': wake_time.hour * 60 + wake_time.minute,
                                'hour_slept_mLight': sleep_time.hour + sleep_time.minute / 60,
                                'hour_woke_up_mLight': wake_time.hour + wake_time.minute / 60
                            })
                            recorded_dates.add(lifelog_date)

                        sleeping = False
                        zero_count = 0
                        first_zero_time = None

            if light > 0:
                zero_count = 0
                first_zero_time = None

    sleep_df = pd.DataFrame(results)

    # 정렬 + 보간
    sleep_df = sleep_df.sort_values(['subject_id', 'lifelog_date'])
    sleep_df['sleep_duration_interp_mLight'] = sleep_df.groupby('subject_id')['sleep_duration_min_mLight'].transform(lambda x: x.interpolate())

    # 시간 단위 파생 컬럼
    sleep_df['sleep_duration_hour_mLight'] = sleep_df['sleep_duration_min_mLight'] / 60
    sleep_df['sleep_duration_interp_hour_mLight'] = sleep_df['sleep_duration_interp_mLight'] / 60

    # 병합
    final = pd.merge(daily_light, sleep_df, on=['subject_id', 'lifelog_date'], how='left')

    return final

In [20]:
def process_mLight2(df):
    from datetime import datetime, timedelta

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])

    # m_light > 0 → m_screen_use로 변환
    df['m_light_on'] = (df['m_light'] > 0).astype(int)

    # base key 확보
    base_keys = df[['subject_id', 'lifelog_date']].drop_duplicates()
    base_keys['lifelog_date'] = base_keys['lifelog_date'].dt.date

    # 밤 9시 ~ 다음날 오전 11시 필터링
    df['hour'] = df['timestamp'].dt.hour
    df = df[(df['hour'] >= 21) | (df['hour'] < 11)].copy()
    df.loc[df['hour'] < 11, 'lifelog_date'] -= pd.Timedelta(days=1)

    df.sort_values(['subject_id', 'timestamp'], inplace=True)

    results = []

    for (subject_id, lifelog_date), group in df.groupby(['subject_id', 'lifelog_date']):
        group = group.sort_values('timestamp').reset_index(drop=True)

        # 1. 중간 각성 제거
        prev = group['m_light_on'].shift(1)
        next_ = group['m_light_on'].shift(-1)
        mask = (group['m_light_on'] == 1) & (prev == 0) & (next_ == 0)
        group.loc[mask, 'm_light_on'] = 0

        # 2. 짧은 각성 블록 제거
        group['is_sleep'] = group['m_light_on'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()
        block_info = group.groupby('block').agg(
            is_sleep=('is_sleep', 'first'),
            size=('is_sleep', 'size')
        )

        for i in range(1, len(block_info) - 1):
            if (
                block_info.iloc[i]['is_sleep'] == False and
                block_info.iloc[i]['size'] <= 2 and
                block_info.iloc[i - 1]['is_sleep'] and
                block_info.iloc[i + 1]['is_sleep']
            ):
                group.loc[group['block'] == block_info.index[i], 'm_light_on'] = 0

        # 3. 수면 블록 추정
        group['is_sleep'] = group['m_light_on'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()
        sleep_blocks = group[group['is_sleep']].groupby('block').agg(
            sleep_start=('timestamp', 'first'),
            sleep_end=('timestamp', 'last'),
            duration_min=('timestamp', lambda x: (x.max() - x.min()).total_seconds() / 60)
        )

        sleep_time = wake_time = duration_min = None
        if not sleep_blocks.empty:
            longest_sleep = sleep_blocks.loc[sleep_blocks['duration_min'].idxmax()]
            sleep_time = longest_sleep['sleep_start'].time()
            wake_time = longest_sleep['sleep_end'].time()
            duration_min = longest_sleep['duration_min']

            # 유효성 조건
            if not (4 <= wake_time.hour < 11):
                wake_time = None
            if not (sleep_time.hour >= 21 or sleep_time.hour < 3):
                sleep_time = None
            if duration_min < 100:
                sleep_time = None
                wake_time = None
                duration_min = None

        results.append({
            'subject_id': subject_id,
            'lifelog_date': lifelog_date.date(),
            'sleep_time': sleep_time,
            'wake_time': wake_time,
            'sleep_duration_min': round(duration_min, 1) if duration_min is not None else None
        })

    sleep_df = pd.DataFrame(results)
    result_df = base_keys.merge(sleep_df, on=['subject_id', 'lifelog_date'], how='left')

    # 시간 → 실수형 숫자 변환
    def time_to_float(t):
        if pd.isna(t):
            return None
        return round(t.hour + t.minute / 60 + t.second / 3600, 4)

    result_df['sleep_time'] = result_df['sleep_time'].apply(time_to_float)
    result_df['wake_time'] = result_df['wake_time'].apply(time_to_float)

    return result_df

In [21]:
def add_ratios(df):
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['weekday'] = df['lifelog_date'].dt.weekday
    df['week_type'] = df['weekday'].apply(lambda x: 'weekend' if x >= 5 else 'weekday')
    df['month'] = df['lifelog_date'].dt.month

    # 평균 계산
    avg_duration = df.groupby(['subject_id', 'month', 'week_type'])['sleep_duration_min'].mean().reset_index(name='avg_sleep_duration')
    sleep_time_avg = df.groupby(['subject_id', 'month', 'week_type'])['sleep_time'].apply(calculate_circular_mean_sleep_time).reset_index(name='avg_sleep_time')
    wake_time_avg = df.groupby(['subject_id', 'month', 'week_type'])['wake_time'].apply(calculate_circular_mean_sleep_time).reset_index(name='avg_wake_time')
    avg_df = sleep_time_avg.merge(wake_time_avg, on=['subject_id', 'month', 'week_type']).merge(avg_duration, on=['subject_id', 'month', 'week_type'])
    df = df.merge(avg_df, on=['subject_id', 'month', 'week_type'], how='left')

    # 비율 및 차이
    df['sleep_time_diff'] = df['avg_sleep_time'] - df['sleep_time']
    df['wake_time_diff'] = df['avg_wake_time'] - df['wake_time']
    df['sleep_duration_diff'] = df['avg_sleep_duration'] - df['sleep_duration_min']
    df['sleep_time_ratio'] = df['sleep_time'] / df['avg_sleep_time']
    df['wake_time_ratio'] = df['wake_time'] / df['avg_wake_time']
    df['sleep_duration_ratio'] = df['sleep_duration_min'] / df['avg_sleep_duration']

    # 정렬 후 lag/변화량
    df = df.sort_values(['subject_id', 'lifelog_date'])
    for lag in [1, 2]:
        df[f'sleep_time_lag{lag}'] = df.groupby('subject_id')['sleep_time'].shift(lag)
        df[f'wake_time_lag{lag}'] = df.groupby('subject_id')['wake_time'].shift(lag)
        df[f'sleep_duration_lag{lag}'] = df.groupby('subject_id')['sleep_duration_min'].shift(lag)
        df[f'sleep_time_diff_lag{lag}'] = df.groupby('subject_id')['sleep_time'].diff(lag)
        df[f'wake_time_diff_lag{lag}'] = df.groupby('subject_id')['wake_time'].diff(lag)
        df[f'sleep_duration_diff_lag{lag}'] = df.groupby('subject_id')['sleep_duration_min'].diff(lag)
    df['week_type_lag1'] = df.groupby('subject_id')['week_type'].shift(1)

    # 이동 평균 (2,3)
    for window in [2, 3]:
        df[f'rolling_sleep_time_{window}d'] = df.groupby('subject_id')['sleep_time'].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)
        df[f'rolling_wake_time_{window}d'] = df.groupby('subject_id')['wake_time'].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)
        df[f'rolling_sleep_duration_{window}d'] = df.groupby('subject_id')['sleep_duration_min'].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)

    # 요일별 평균 수면 비교
    weekday_avg = df.groupby(['subject_id', 'weekday'])['sleep_duration_min'].mean().reset_index(name='weekday_avg_sleep')
    df = df.merge(weekday_avg, on=['subject_id', 'weekday'], how='left')
    df['sleep_duration_vs_weekday_avg'] = df['sleep_duration_min'] - df['weekday_avg_sleep']

    # 급격한 수면시간 변화 여부 (60분 이상 변화)
    df['is_sleep_duration_change_large'] = (df['sleep_duration_diff_lag1'].abs() > 60).astype(int)

    return df

In [22]:
mLight21 = process_mLight(mLight)

# check
print(f'\n # mLight21 shape: {mLight21.shape}')
mLight21.head(1)

Processing light-based sleep detection:   0%|          | 0/10 [00:00<?, ?it/s]


 # mLight21 shape: (700, 17)


,subject_id,lifelog_date,light_mean,light_std,light_max,light_min,light_night_mean,light_day_mean,light_night_ratio,sleep_duration_min_mLight,sleep_time_min_mLight,wake_time_min_mLight,hour_slept_mLight,hour_woke_up_mLight,sleep_duration_interp_mLight,sleep_duration_hour_mLight,sleep_duration_interp_hour_mLight
0,id01,2024-06-26,364.5068,395.6594,1886.0000,0.0000,184.9231,403.4167,0.1781,340.0000,1409.0000,309.0000,23.4833,5.1500,340.0000,5.6667,5.6667


In [23]:
mLight22 = process_mLight2(mLight)
mLight22['sleep_time'] = np.where(mLight22['sleep_time']<10,mLight22['sleep_time']+24,mLight22['sleep_time'])                          ### 수정
mLight22['sleep_duration_min'] = mLight22.apply(lambda x: calculate_sleep_duration_min(x['sleep_time'],x['wake_time']),axis=1)
mLight22 = add_ratios(mLight22)
mLight22 = mLight22.drop(columns=['week_type','wake_time_lag1'])
mLight22.columns = ['subject_id', 'lifelog_date']+['light_'+i for i in mLight22.columns if i not in ['subject_id', 'lifelog_date']]
mLight22['lifelog_date'] = mLight22['lifelog_date'].astype(str)

# check
# mLight22 shape: (700, 55)
print(f'\n # mLight22 shape: {mLight22.shape}')
mLight22.head(1)


 # mLight22 shape: (700, 37)


,subject_id,lifelog_date,light_sleep_time,light_wake_time,light_sleep_duration_min,light_weekday,light_month,light_avg_sleep_time,light_avg_wake_time,light_avg_sleep_duration,light_sleep_time_diff,light_wake_time_diff,light_sleep_duration_diff,light_sleep_time_ratio,light_wake_time_ratio,light_sleep_duration_ratio,light_sleep_time_lag1,light_sleep_duration_lag1,light_sleep_time_diff_lag1,light_wake_time_diff_lag1,light_sleep_duration_diff_lag1,light_sleep_time_lag2,light_wake_time_lag2,light_sleep_duration_lag2,light_sleep_time_diff_lag2,light_wake_time_diff_lag2,light_sleep_duration_diff_lag2,light_week_type_lag1,light_rolling_sleep_time_2d,light_rolling_wake_time_2d,light_rolling_sleep_duration_2d,light_rolling_sleep_time_3d,light_rolling_wake_time_3d,light_rolling_sleep_duration_3d,light_weekday_avg_sleep,light_sleep_duration_vs_weekday_avg,light_is_sleep_duration_change_large
0,id01,2024-06-26,23.4833,4.9833,330.0000,2,6,23.3722,5.0944,343.3333,-0.1111,0.1111,13.3333,1.0048,0.9782,0.9612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.4833,4.9833,330.0000,23.4833,4.9833,330.0000,358.2000,-28.2000,0


In [24]:
def estimate_lights_off_time(df, light_threshold=2):

    # 시간 → 실수형 (예: 23:30 → 23.5)
    def time_to_float(t):
        if pd.isna(t):
            return None
        return round(t.hour + t.minute / 60 + t.second / 3600, 4)

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])

    # 밤 시간대 필터 (21시~23시 or 0~3시)
    df['hour'] = df['timestamp'].dt.hour
    df = df[(df['hour'] >= 21) | (df['hour'] <= 3)].copy()

    # 자정 이후는 전날 night로 처리
    df.loc[df['hour'] <= 3, 'lifelog_date'] -= pd.Timedelta(days=1)

    # 낮은 조도 조건
    df = df[df['m_light'] <= light_threshold]

    # 각 (subject_id, lifelog_date)별 불 끈 시각 추출
    lights_off_df = (
        df.groupby(['subject_id', 'lifelog_date'])['timestamp']
        .min()
        .reset_index(name='lights_off_time')
    )

    # 실수형 시각으로 변환
    lights_off_df['lights_off_time'] = lights_off_df['lights_off_time'].dt.time.apply(time_to_float)

    return lights_off_df

In [25]:
mLight23 = estimate_lights_off_time(mLight)
mLight23['lights_off_time'] = np.where(mLight23['lights_off_time']<24,mLight23['lights_off_time']+24,mLight23['lights_off_time'])
mLight23.head()

,subject_id,lifelog_date,lights_off_time
0,id01,2024-06-26,45.0500
1,id01,2024-06-27,45.1500
2,id01,2024-06-28,47.1500
3,id01,2024-06-29,45.9833
4,id01,2024-06-30,45.1500


### ✔️ mScreenStatus 화면 사용여부

- Indicates whether the smartphone screen is in use.
 - 기상시간, 취침시간, 수면시간
 - 휴대폰 이용횟수, 이용시간
 - 00 - 05 사이에 휴대폰 이용한 건수

In [26]:
mScreenStatus['lifelog_date'] = mScreenStatus['timestamp'].astype(str).str[:10]
mScreenStatus.head(1)

,subject_id,timestamp,m_screen_use,lifelog_date
0,id01,2024-06-26 12:03:00,0,2024-06-26


In [27]:
def preprocess_mScreenStatus(df):
    from datetime import datetime, time as dtime, timedelta

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])

    # base key 확보
    base_keys = df[['subject_id', 'lifelog_date']].drop_duplicates()
    base_keys['lifelog_date'] = base_keys['lifelog_date'].dt.date

    # 밤 9시부터 다음날 오전 11시 필터링
    df['hour'] = df['timestamp'].dt.hour
    df = df[(df['hour'] >= 21) | (df['hour'] < 11)].copy()
    df.loc[df['hour'] < 11, 'lifelog_date'] -= pd.Timedelta(days=1)

    df.sort_values(['subject_id', 'timestamp'], inplace=True)

    results = []

    for (subject_id, lifelog_date), group in df.groupby(['subject_id', 'lifelog_date']):
        group = group.sort_values('timestamp').reset_index(drop=True)

        # 1. 중간 각성(앞뒤 0, 본인 1) 제거
        prev = group['m_screen_use'].shift(1)
        next_ = group['m_screen_use'].shift(-1)
        mask = (group['m_screen_use'] == 1) & (prev == 0) & (next_ == 0)
        group.loc[mask, 'm_screen_use'] = 0

        # 2. 블록 단위로 짧은 각성 블록 제거
        group['is_sleep'] = group['m_screen_use'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()
        block_info = group.groupby('block').agg(
            is_sleep=('is_sleep', 'first'),
            size=('is_sleep', 'size')
        )

        for i in range(1, len(block_info) - 1):
            if (
                block_info.iloc[i]['is_sleep'] == False and
                block_info.iloc[i]['size'] <= 2 and
                block_info.iloc[i - 1]['is_sleep'] and
                block_info.iloc[i + 1]['is_sleep']
            ):
                group.loc[group['block'] == block_info.index[i], 'm_screen_use'] = 0

        # 다시 블록 재계산 후 수면 추정
        group['is_sleep'] = group['m_screen_use'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()
        sleep_blocks = group[group['is_sleep']].groupby('block').agg(
            sleep_start=('timestamp', 'first'),
            sleep_end=('timestamp', 'last'),
            duration_min=('timestamp', lambda x: (x.max() - x.min()).total_seconds() / 60)
        )

        sleep_time = wake_time = duration_min = None
        if not sleep_blocks.empty:
            longest_sleep = sleep_blocks.loc[sleep_blocks['duration_min'].idxmax()]
            sleep_time = longest_sleep['sleep_start'].time()
            wake_time = longest_sleep['sleep_end'].time()
            duration_min = (
                datetime.combine(datetime.today(), wake_time) - datetime.combine(datetime.today(), sleep_time)
            ).total_seconds() / 60
            if duration_min < 0:
                duration_min += 1440

            if not (4 <= wake_time.hour < 11):
                wake_time = None
            if not (sleep_time.hour >= 21 or sleep_time.hour < 3):
                sleep_time = None
            if duration_min < 100:
                sleep_time = None
                wake_time = None
                duration_min = None

        results.append({
            'subject_id': subject_id,
            'lifelog_date': lifelog_date.date(),
            'sleep_time': sleep_time,
            'wake_time': wake_time,
            'sleep_duration_min': round(duration_min, 1) if duration_min is not None else None
        })


    sleep_df = pd.DataFrame(results)
    result_df = base_keys.merge(sleep_df, on=['subject_id', 'lifelog_date'], how='left')

    # 시간 → 실수형 숫자 (예: 23:30 → 23.5)
    def time_to_float(t):
        if pd.isna(t):
            return None
        return round(t.hour + t.minute / 60 + t.second / 3600, 4)

    result_df['sleep_time'] = result_df['sleep_time'].apply(time_to_float)
    result_df['wake_time'] = result_df['wake_time'].apply(time_to_float)

    return result_df

In [28]:
def preprocess_mScreenStatus(df):
    from datetime import datetime, timedelta

    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])

    base_keys = df[['subject_id', 'lifelog_date']].drop_duplicates()
    base_keys['lifelog_date'] = base_keys['lifelog_date'].dt.date

    # 밤 9시 ~ 다음날 오전 11시 필터링
    df['hour'] = df['timestamp'].dt.hour
    df = df[(df['hour'] >= 21) | (df['hour'] < 11)].copy()
    df.loc[df['hour'] < 11, 'lifelog_date'] -= pd.Timedelta(days=1)
    df.sort_values(['subject_id', 'timestamp'], inplace=True)

    results = []

    for (subject_id, lifelog_date), group in df.groupby(['subject_id', 'lifelog_date']):
        group = group.sort_values('timestamp').reset_index(drop=True)

        # 중간 각성 제거
        prev = group['m_screen_use'].shift(1)
        next_ = group['m_screen_use'].shift(-1)
        mask = (group['m_screen_use'] == 1) & (prev == 0) & (next_ == 0)
        group.loc[mask, 'm_screen_use'] = 0

        # 짧은 각성 블록 제거
        group['is_sleep'] = group['m_screen_use'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()
        block_info = group.groupby('block').agg(
            is_sleep=('is_sleep', 'first'),
            size=('is_sleep', 'size')
        )

        for i in range(1, len(block_info) - 1):
            if (
                block_info.iloc[i]['is_sleep'] == False and
                block_info.iloc[i]['size'] <= 2 and
                block_info.iloc[i - 1]['is_sleep'] and
                block_info.iloc[i + 1]['is_sleep']
            ):
                group.loc[group['block'] == block_info.index[i], 'm_screen_use'] = 0

        # 블록 재계산
        group['is_sleep'] = group['m_screen_use'] == 0
        group['block'] = (group['is_sleep'] != group['is_sleep'].shift()).cumsum()

        sleep_blocks = group[group['is_sleep']].groupby('block').agg(
            sleep_start=('timestamp', 'first'),
            sleep_end=('timestamp', 'last'),
            duration_min=('timestamp', lambda x: (x.max() - x.min()).total_seconds() / 60)
        )

        sleep_time = wake_time = duration_min = None
        if not sleep_blocks.empty:
            longest_sleep = sleep_blocks.loc[sleep_blocks['duration_min'].idxmax()]
            sleep_time = longest_sleep['sleep_start'].time()
            wake_time = longest_sleep['sleep_end'].time()
            duration_min = longest_sleep['duration_min']  # ✅ 정확하게 자정 넘는 경우도 반영됨

            # 유효 시간 범위 조건
            if not (4 <= wake_time.hour < 11):
                wake_time = None
            if not (sleep_time.hour >= 21 or sleep_time.hour < 3):
                sleep_time = None
            if duration_min < 100:
                sleep_time = None
                wake_time = None
                duration_min = None

        results.append({
            'subject_id': subject_id,
            'lifelog_date': lifelog_date.date(),
            'sleep_time': sleep_time,
            'wake_time': wake_time,
            'sleep_duration_min': round(duration_min, 1) if duration_min is not None else None
        })

    sleep_df = pd.DataFrame(results)
    result_df = base_keys.merge(sleep_df, on=['subject_id', 'lifelog_date'], how='left')

    # 시간 → 실수형 숫자 변환
    def time_to_float(t):
        if pd.isna(t):
            return None
        return round(t.hour + t.minute / 60 + t.second / 3600, 4)

    result_df['sleep_time'] = result_df['sleep_time'].apply(time_to_float)
    result_df['wake_time'] = result_df['wake_time'].apply(time_to_float)

    return result_df

In [29]:
def add_ratios(df):
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['weekday'] = df['lifelog_date'].dt.weekday  # 0=월 ~ 6=일
    df['week_type'] = df['weekday'].apply(lambda x: 'weekend' if x >= 5 else 'weekday')
    df['month'] = df['lifelog_date'].dt.month

    # 평균 계산
    avg_duration = df.groupby(['subject_id', 'month', 'week_type'])['sleep_duration_min'].mean().reset_index(name='avg_sleep_duration')
    sleep_time_avg = df.groupby(['subject_id', 'month', 'week_type'])['sleep_time'].apply(calculate_circular_mean_sleep_time).reset_index(name='avg_sleep_time')
    wake_time_avg = df.groupby(['subject_id', 'month', 'week_type'])['wake_time'].apply(calculate_circular_mean_sleep_time).reset_index(name='avg_wake_time')
    avg_df = sleep_time_avg.merge(wake_time_avg, on=['subject_id', 'month', 'week_type']).merge(avg_duration, on=['subject_id', 'month', 'week_type'])
    df = df.merge(avg_df, on=['subject_id', 'month', 'week_type'], how='left')

    # 비율 변수
    df['sleep_time_diff'] = df['avg_sleep_time'] - df['sleep_time']
    df['wake_time_diff'] = df['avg_wake_time'] - df['wake_time']
    df['sleep_duration_diff'] = df['avg_sleep_duration'] - df['sleep_duration_min']
    df['sleep_time_ratio'] = df['sleep_time'] / df['avg_sleep_time']
    df['wake_time_ratio'] = df['wake_time'] / df['avg_wake_time']
    df['sleep_duration_ratio'] = df['sleep_duration_min'] / df['avg_sleep_duration']

    # 정렬 및 lag/변화량
    df = df.sort_values(['subject_id', 'lifelog_date'])
    for lag in [1, 2]:
        df[f'sleep_time_lag{lag}'] = df.groupby('subject_id')['sleep_time'].shift(lag)
        df[f'wake_time_lag{lag}'] = df.groupby('subject_id')['wake_time'].shift(lag)
        df[f'sleep_duration_lag{lag}'] = df.groupby('subject_id')['sleep_duration_min'].shift(lag)
        df[f'sleep_time_diff_lag{lag}'] = df.groupby('subject_id')['sleep_time'].diff(lag)
        df[f'wake_time_diff_lag{lag}'] = df.groupby('subject_id')['wake_time'].diff(lag)
        df[f'sleep_duration_diff_lag{lag}'] = df.groupby('subject_id')['sleep_duration_min'].diff(lag)
    df['week_type_lag1'] = df.groupby('subject_id')['week_type'].shift(1)

    # 이동 평균
    for window in [2, 3]:
        df[f'rolling_sleep_time_{window}d'] = df.groupby('subject_id')['sleep_time'].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)
        df[f'rolling_wake_time_{window}d'] = df.groupby('subject_id')['wake_time'].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)
        df[f'rolling_sleep_duration_{window}d'] = df.groupby('subject_id')['sleep_duration_min'].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)

    # 요일별 평균 수면 비교
    weekday_avg = df.groupby(['subject_id', 'weekday'])['sleep_duration_min'].mean().reset_index(name='weekday_avg_sleep')
    df = df.merge(weekday_avg, on=['subject_id', 'weekday'], how='left')
    df['sleep_duration_vs_weekday_avg'] = df['sleep_duration_min'] - df['weekday_avg_sleep']

    # 급격한 수면시간 변화 여부
    df['is_sleep_duration_change_large'] = (df['sleep_duration_diff_lag1'].abs() > 60).astype(int)

    return df

def calculate_circular_mean_sleep_time(series):
    radians = series * 2 * np.pi / 1440  # 분 단위 -> 라디안
    sin_sum = np.sin(radians).sum()
    cos_sum = np.cos(radians).sum()
    mean_angle = np.arctan2(sin_sum, cos_sum)
    if mean_angle < 0:
        mean_angle += 2 * np.pi
    return mean_angle * 1440 / (2 * np.pi)

In [30]:
mScreenStatus2 = preprocess_mScreenStatus(mScreenStatus)
mScreenStatus2['sleep_time'] = np.where(mScreenStatus2['sleep_time']<10,mScreenStatus2['sleep_time']+24,mScreenStatus2['sleep_time'])         ### 수정
mScreenStatus2['sleep_duration_min'] = mScreenStatus2.apply(lambda x: calculate_sleep_duration_min(x['sleep_time'],x['wake_time']),axis=1)
mScreenStatus2 = add_ratios(mScreenStatus2)

# check
print(f'\n # mScreenStatus2 shape: {mScreenStatus2.shape}')
mScreenStatus2.head(1)


 # mScreenStatus2 shape: (700, 39)


,subject_id,lifelog_date,sleep_time,wake_time,sleep_duration_min,weekday,week_type,month,avg_sleep_time,avg_wake_time,avg_sleep_duration,sleep_time_diff,wake_time_diff,sleep_duration_diff,sleep_time_ratio,wake_time_ratio,sleep_duration_ratio,sleep_time_lag1,wake_time_lag1,sleep_duration_lag1,sleep_time_diff_lag1,wake_time_diff_lag1,sleep_duration_diff_lag1,sleep_time_lag2,wake_time_lag2,sleep_duration_lag2,sleep_time_diff_lag2,wake_time_diff_lag2,sleep_duration_diff_lag2,week_type_lag1,rolling_sleep_time_2d,rolling_wake_time_2d,rolling_sleep_duration_2d,rolling_sleep_time_3d,rolling_wake_time_3d,rolling_sleep_duration_3d,weekday_avg_sleep,sleep_duration_vs_weekday_avg,is_sleep_duration_change_large
0,id01,2024-06-26,23.4500,5.2500,348.0000,2,weekday,6,23.1944,5.4889,377.6667,-0.2556,0.2389,29.6667,1.0110,0.9565,0.9214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.4500,5.2500,348.0000,23.4500,5.2500,348.0000,377.0000,-29.0000,0


In [31]:
mScreenStatus2평균수면시간 = mScreenStatus2.groupby(['subject_id','week_type']).apply(lambda x: pd.Series({
     '평균 취침시간':circular_mean_sleep_time(x['sleep_time'])
    ,'평균 기상시간':circular_mean_sleep_time(x['wake_time'])
    ,'평균 수면시간':x['sleep_duration_min'].mean()
})).reset_index()

# 저장
fname = f'mScreenStatus2평균수면시간.xlsx'
print(fname)
mScreenStatus2평균수면시간.to_excel(fname, index=False)

# check
mScreenStatus2평균수면시간.head()

mScreenStatus2평균수면시간.xlsx


,subject_id,week_type,평균 취침시간,평균 기상시간,평균 수면시간
0,id01,weekday,22:46,05:54,424.5135
1,id01,weekend,22:25,06:04,456.7059
2,id02,weekday,22:57,07:14,492.5778
3,id02,weekend,23:13,07:27,494.9583
4,id03,weekday,00:30,09:08,494.4286


### ✔️ mUsageStats 앱사용통계
- mUsageStats: Indicates which apps were used on the smartphone and for how long.


In [32]:
def extract_mUsageStats_info(row):
    m_data = row['m_usage_stats']
    app_name = [item['app_name'] for item in m_data]
    total_time = [item['total_time'] for item in m_data]
    return pd.Series({'app_name': app_name, 'total_time': total_time})

mUsageStats[['app_name', 'total_time']] = mUsageStats.apply(extract_mUsageStats_info, axis=1)
mUsageStats['lifelog_date'] = mUsageStats['timestamp'].astype(str).str[:10]
# mUsageStats = fill_missing_dates_by_subject(mUsageStats)
mUsageStats.head(1)

,subject_id,timestamp,m_usage_stats,app_name,total_time,lifelog_date
0,id01,2024-06-26 13:00:00,"[{'app_name': ' 캐시워크', 'total_time': 69}, {'app_name': 'NAVER', 'total_time': 549}, {'app_name': ' ✝️성경일독Q', 'total_time': 7337}]","[ 캐시워크, NAVER, ✝️성경일독Q]","[69, 549, 7337]",2024-06-26


In [33]:
def process_mUsageStats(df):
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['요일'] = df['lifelog_date'].dt.day_name()

    # 리스트 평탄화
    exploded_df = df.explode(['app_name', 'total_time'])
    exploded_df['total_time'] = exploded_df['total_time'].astype(float)
    exploded_df['total_time'] = exploded_df['total_time'] * 0.001 / 60  # 밀리초 → 초 → 분 변환

    # app_name 특수문자 제거
    exploded_df['app_name'] = exploded_df['app_name'].astype(str).apply(
        lambda x: re.sub(r'[^가-힣a-zA-Z0-9]', '', x)
    )

    # 시스템 앱 제거
    filtered_df = exploded_df[~exploded_df['app_name'].isin(['시스템UI'])]  # '시스템UI'만 제거 (OneUI홈은 포함)

    # 주요 파생변수 생성
    def calculate_daily_metrics(group):
        last_use = group['timestamp'].max()

        app_times = {
            '통화_time': group[group['app_name'] == '통화']['total_time'].sum(),
            '전화_time': group[group['app_name'] == '전화']['total_time'].sum(),
            'YouTube_time': group[group['app_name'] == 'YouTube']['total_time'].sum(),
            '메신저_time': group[group['app_name'].isin(['메시지', '카카오톡'])]['total_time'].sum(),
            'NAVER_time': group[group['app_name'] == 'NAVER']['total_time'].sum(),
            '캐시워크_time': group[group['app_name'] == '캐시워크']['total_time'].sum(),
            '성경일독Q_time': group[group['app_name'] == '성경일독Q']['total_time'].sum(),
            'OneUI홈_time': group[group['app_name'] == 'OneUI홈']['total_time'].sum(),
        }

        return pd.Series({
            **app_times,
            'unique_app_count': group['app_name'].nunique(),
            'total_screen_time': group['total_time'].sum()
        })

    # daily metrics 생성
    daily_stats = filtered_df.groupby(['subject_id','lifelog_date']).apply(calculate_daily_metrics).reset_index()

    # subject_id별 평균 총화면시간 구하기
    avg_screen_time = daily_stats.groupby('subject_id')['total_screen_time'].mean().to_dict()

    # 평균대비 화면사용량(%) 생성
    def compute_screen_usage(row):
        avg_time = avg_screen_time.get(row['subject_id'], np.nan)
        if pd.isna(avg_time) or avg_time == 0:
            return np.nan
        return round((row['total_screen_time'] / avg_time - 1) * 100, 1)

    daily_stats['screen_time_vs_avg_pct'] = daily_stats.apply(compute_screen_usage, axis=1)

    return daily_stats

In [34]:
def process_mUsageStats(df):
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['요일'] = df['lifelog_date'].dt.day_name()
    df['hour'] = df['timestamp'].dt.hour

    # 시간대 분류
    def map_time_period(row):
        if 20 <= row['hour'] <= 23:
            return 'beforebed'
        else:
            return 'activehour'

    df['time_period'] = df.apply(map_time_period, axis=1)

    # 리스트 평탄화
    exploded_df = df.explode(['app_name', 'total_time'])
    exploded_df['total_time'] = exploded_df['total_time'].astype(float)
    exploded_df['total_time'] = exploded_df['total_time'] * 0.001 / 60  # 밀리초 → 초 → 분 변환

    # app_name 특수문자 제거
    exploded_df['app_name'] = exploded_df['app_name'].astype(str).apply(
        lambda x: re.sub(r'[^가-힣a-zA-Z0-9]', '', x)
    )

    # 시스템 앱 제거
    filtered_df = exploded_df[~exploded_df['app_name'].isin(['시스템UI'])]

    # 주요 파생변수 생성
    def calculate_daily_metrics(group):
        app_times = {
            '통화_time': group[group['app_name'] == '통화']['total_time'].sum(),
            '전화_time': group[group['app_name'] == '전화']['total_time'].sum(),
            'YouTube_time': group[group['app_name'] == 'YouTube']['total_time'].sum(),
            '메신저_time': group[group['app_name'].isin(['메시지', '카카오톡'])]['total_time'].sum(),
            'NAVER_time': group[group['app_name'] == 'NAVER']['total_time'].sum(),
            '캐시워크_time': group[group['app_name'] == '캐시워크']['total_time'].sum(),
            '성경일독Q_time': group[group['app_name'] == '성경일독Q']['total_time'].sum(),
            'OneUI홈_time': group[group['app_name'] == 'OneUI홈']['total_time'].sum(),
        }

        return pd.Series({
            **app_times,
            'unique_app_count': group['app_name'].nunique(),
            'total_screen_time': group['total_time'].sum()
        })

    # 일자/시간대별 요약
    daily_stats = filtered_df.groupby(['subject_id', 'lifelog_date', 'time_period']).apply(calculate_daily_metrics).reset_index()

    # subject_id별 평균 총화면시간
    avg_screen_time = daily_stats.groupby('subject_id')['total_screen_time'].mean().to_dict()

    # 평균 대비 비율
    def compute_screen_usage(row):
        avg_time = avg_screen_time.get(row['subject_id'], np.nan)
        if pd.isna(avg_time) or avg_time == 0:
            return np.nan
        return round((row['total_screen_time'] / avg_time - 1) * 100, 1)

    daily_stats['screen_time_vs_avg_pct'] = daily_stats.apply(compute_screen_usage, axis=1)

    # 피벗
    daily_stats = daily_stats.pivot(index=['subject_id', 'lifelog_date'], columns='time_period')
    daily_stats.columns = [f"{tp}_{metric}" for metric, tp in daily_stats.columns]
    daily_stats = daily_stats.reset_index()

    return daily_stats

In [35]:
mUsageStats2 = process_mUsageStats(mUsageStats)

# check
print(f'\n # mUsageStats2 shape: {mUsageStats2.shape}')
mUsageStats2.head(1)


 # mUsageStats2 shape: (689, 24)


,subject_id,lifelog_date,activehour_통화_time,beforebed_통화_time,activehour_전화_time,beforebed_전화_time,activehour_YouTube_time,beforebed_YouTube_time,activehour_메신저_time,beforebed_메신저_time,activehour_NAVER_time,beforebed_NAVER_time,activehour_캐시워크_time,beforebed_캐시워크_time,activehour_성경일독Q_time,beforebed_성경일독Q_time,activehour_OneUI홈_time,beforebed_OneUI홈_time,activehour_unique_app_count,beforebed_unique_app_count,activehour_total_screen_time,beforebed_total_screen_time,activehour_screen_time_vs_avg_pct,beforebed_screen_time_vs_avg_pct
0,id01,2024-06-26,9.0010,0.2079,11.3007,0.7731,0.1061,0.0000,43.6359,14.5713,8.4852,0.1351,18.6694,5.4722,88.3836,27.6892,61.1160,27.9861,25.0000,20.0000,266.7672,156.8681,-29.0000,-58.3000


### ✔️ mWifi 주변wifi 정보
- Wifi devices around individual subject.
 - -30 ~ -50 dBm	매우 강한 신호 (최적)
 - -51 ~ -60 dBm	강한 신호 (문제 없음)
 - -61 ~ -70 dBm	괜찮은 신호 (약간 느릴 수 있음)
 - -71 ~ -80 dBm	약한 신호 (끊김 주의)
 - -81 dBm 이하	매우 약한 신호 (거의 끊김)

In [36]:
def extract_wifi_info(row):
    wifi_data = row['m_wifi']
    bssids = [item['bssid'] for item in wifi_data]
    rssis = [item['rssi'] for item in wifi_data]
    return pd.Series({'bssid': bssids, 'rssi': rssis})

mWifi = pd.read_parquet(f'{PATH}/ETRI_lifelog_dataset/ch2025_data_items/ch2025_mWifi.parquet')
mWifi[['bssid', 'rssi']] = mWifi.apply(extract_wifi_info, axis=1)
mWifi['lifelog_date'] = mWifi['timestamp'].astype(str).str[:10]
# mWifi = fill_missing_dates_by_subject(mWifi)
mWifi.head(1)

,subject_id,timestamp,m_wifi,bssid,rssi,lifelog_date
0,id01,2024-06-26 12:03:00,"[{'bssid': 'a0:0f:37:9a:5d:8b', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8c', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8d', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8e', 'rssi': -78}, {'bssid': 'a0:0f:37:9a:5d:8f', 'rssi': -78}, {'bssid': 'a0:0f:37:96:56:ef', 'rssi': -58}, {'bssid': '88:36:6c:86:75:84', 'rssi': -72}, {'bssid': 'a0:0f:37:96:56:ee', 'rssi': -58}, {'bssid': 'a0:0f:37:96:56:ed', 'rssi': -58}, {'bssid': '86:25:19:b5:b2:a5', 'rssi': -61}, {'bssid': 'a0:0f:37:96:56:ec', 'rssi': -58}, {'bssid': '1e:39:29:8e:fb:e9', 'rssi': -71}, {'bssid': '52:c2:e8:c7:9b:e4', 'rssi': -82}, {'bssid': 'a0:0f:37:96:56:eb', 'rssi': -58}, {'bssid': '12:e3:c7:09:20:34', 'rssi': -88}, {'bssid': '58:86:94:4a:08:b8', 'rssi': -82}, {'bssid': '90:9f:33:28:d0:2e', 'rssi': -78}, {'bssid': '00:26:66:bc:4e:18', 'rssi': -85}, {'bssid': 'f6:0a:f4:43:4b:ba', 'rssi': -45}, {'bssid': '10:e3:c7:09:20:35', 'rssi': -63}, {'bssid': '10:e3:c7:09:20:34', 'rssi': -89}, {'bssid': '1c:39:29:48:04:92', 'rssi': -82}, {'bssid': '12:e3:c7:07:9d:df', 'rssi': -83}, {'bssid': '86:25:19:c3:44:07', 'rssi': -84}, {'bssid': 'a0:0f:37:9a:37:2f', 'rssi': -76}, {'bssid': 'a0:0f:37:9a:37:2e', 'rssi': -76}, {'bssid': 'a0:0f:37:9a:37:2d', 'rssi': -76}, {'bssid': '0a:09:b4:74:05:ec', 'rssi': -72}, {'bssid': 'a0:0f:37:9a:37:2c', 'rssi': -76}, {'bssid': 'a0:0f:37:9a:37:2b', 'rssi': -76}, {'bssid': '0a:09:b4:74:05:eb', 'rssi': -59}, {'bssid': 'c0:25:2f:d8:c1:a6', 'rssi': -82}, {'bssid': '16:7f:67:bb:fa:f8', 'rssi': -79}, {'bssid': '3c:f3:92:ff:00:01', 'rssi': -82}, {'bssid': '06:09:b4:74:05:ec', 'rssi': -72}, {'bssid': '06:09:b4:74:05:eb', 'rssi': -59}, {'bssid': '12:e3:c7:0a:74:d1', 'rssi': -78}, {'bssid': '88:36:6c:a9:6f:8e', 'rssi': -63}, {'bssid': '02:e3:c7:09:20:34', 'rssi': -88}, {'bssid': '00:09:b4:74:05:eb', 'rssi': -60}, {'bssid': '00:09:b4:74:05:ec', 'rssi': -72}, {'bssid': '00:1d:93:93:cf:fe', 'rssi': -19}, {'bssid': '8e:e2:ac:a5:9d:15', 'rssi': -72}]","[a0:0f:37:9a:5d:8b, a0:0f:37:9a:5d:8c, a0:0f:37:9a:5d:8d, a0:0f:37:9a:5d:8e, a0:0f:37:9a:5d:8f, a0:0f:37:96:56:ef, 88:36:6c:86:75:84, a0:0f:37:96:56:ee, a0:0f:37:96:56:ed, 86:25:19:b5:b2:a5, a0:0f:37:96:56:ec, 1e:39:29:8e:fb:e9, 52:c2:e8:c7:9b:e4, a0:0f:37:96:56:eb, 12:e3:c7:09:20:34, 58:86:94:4a:08:b8, 90:9f:33:28:d0:2e, 00:26:66:bc:4e:18, f6:0a:f4:43:4b:ba, 10:e3:c7:09:20:35, 10:e3:c7:09:20:34, 1c:39:29:48:04:92, 12:e3:c7:07:9d:df, 86:25:19:c3:44:07, a0:0f:37:9a:37:2f, a0:0f:37:9a:37:2e, a0:0f:37:9a:37:2d, 0a:09:b4:74:05:ec, a0:0f:37:9a:37:2c, a0:0f:37:9a:37:2b, 0a:09:b4:74:05:eb, c0:25:2f:d8:c1:a6, 16:7f:67:bb:fa:f8, 3c:f3:92:ff:00:01, 06:09:b4:74:05:ec, 06:09:b4:74:05:eb, 12:e3:c7:0a:74:d1, 88:36:6c:a9:6f:8e, 02:e3:c7:09:20:34, 00:09:b4:74:05:eb, 00:09:b4:74:05:ec, 00:1d:93:93:cf:fe, 8e:e2:ac:a5:9d:15]","[-78, -78, -78, -78, -78, -58, -72, -58, -58, -61, -58, -71, -82, -58, -88, -82, -78, -85, -45, -63, -89, -82, -83, -84, -76, -76, -76, -72, -76, -76, -59, -82, -79, -82, -72, -59, -78, -63, -88, -60, -72, -19, -72]",2024-06-26


In [37]:
def process_mWifi(df,threshold):

    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['요일'] = df['lifelog_date'].dt.day_name()
    df['hour'] = df['timestamp'].dt.hour

    # 시간대 분류
    def map_time_period(row):
        if 20 <= row['hour'] <= 23:
            return 'beforebed'
        else:
            return 'activehour'

    df['time_period'] = df.apply(map_time_period, axis=1)

    features = []
    grouped = df.groupby(['subject_id', 'lifelog_date','time_period'])

    for (subject_id, date, period), group in grouped:
        scan_count = len(group)
        bssid_flat = sum(group['bssid'], [])  # flatten
        rssi_flat = sum(group['rssi'], [])    # flatten

        unique_bssid_count = len(set(bssid_flat))
        avg_rssi = sum(rssi_flat) / len(rssi_flat) if rssi_flat else None
        max_rssi = max(rssi_flat) if rssi_flat else None
        min_rssi = min(rssi_flat) if rssi_flat else None
        strong_rssi_ratio = sum(1 for r in rssi_flat if r > -60) / len(rssi_flat) if rssi_flat else 0
        empty_scan_count = sum(1 for b in group['bssid'] if len(b) == 0)

        # 가장 많이 탐지된 BSSID
        bssid_counter = Counter(bssid_flat)
        top_bssid, top_bssid_count = bssid_counter.most_common(1)[0] if bssid_counter else (None, 0)

        first_time = group['timestamp'].min()
        last_time = group['timestamp'].max()
        hour_span = (last_time - first_time).total_seconds() / 60  # 분 단위

        features.append({
            'subject_id': subject_id,
            'lifelog_date': date,
            'time_period': period,  #
            'scan_count': scan_count,
            'unique_bssid_count': unique_bssid_count,
            'avg_rssi': avg_rssi,
            'max_rssi': max_rssi,
            'min_rssi': min_rssi,
            'strong_signal_ratio': strong_rssi_ratio,
            'empty_scan_count': empty_scan_count,
            'top_bssid': top_bssid,
            'top_bssid_count': top_bssid_count,
            'hour_span_minutes': hour_span
        })

    daily_stats = pd.DataFrame(features)

    # 피벗
    daily_stats = daily_stats.pivot(index=['subject_id', 'lifelog_date'], columns='time_period')
    daily_stats.columns = [f"{tp}_{metric}" for metric, tp in daily_stats.columns]
    daily_stats = daily_stats.reset_index()

    return daily_stats

In [38]:
mWifi2 = process_mWifi(mWifi,threshold=-60)

# check
print(f'\n # mWifi2 shape: {mWifi2.shape}')
mWifi2.head(1)


 # mWifi2 shape: (685, 22)


,subject_id,lifelog_date,activehour_scan_count,beforebed_scan_count,activehour_unique_bssid_count,beforebed_unique_bssid_count,activehour_avg_rssi,beforebed_avg_rssi,activehour_max_rssi,beforebed_max_rssi,activehour_min_rssi,beforebed_min_rssi,activehour_strong_signal_ratio,beforebed_strong_signal_ratio,activehour_empty_scan_count,beforebed_empty_scan_count,activehour_top_bssid,beforebed_top_bssid,activehour_top_bssid_count,beforebed_top_bssid_count,activehour_hour_span_minutes,beforebed_hour_span_minutes
0,id01,2024-06-26,48.0000,21.0000,354.0000,56.0000,-71.7103,-64.5025,-19.0000,-26.0000,-91.0000,-88.0000,0.1892,0.3881,0.0000,0.0000,86:25:19:9f:9b:be,04:09:a5:3a:c8:6a,19.0000,15.0000,470.0000,236.0000


### ✔️ wHr 심박동수
- Heart rate readings recorded by the smartwatch.


In [39]:
def process_wHr(df):
    heart_rate = df["heart_rate"].values  # [[0, 1, 2, ...], ...]

    def _process_feature(heart_rate):
        if len(heart_rate) == 0:
            return 0., 0., 0., 0., 0.

        heart_rate = np.array(sum(map(lambda x: x.tolist(), heart_rate), []))
        mean_hr = heart_rate.mean() if len(heart_rate) > 0 else 0
        min_hr = heart_rate.min() if len(heart_rate) > 0 else 0
        max_hr = heart_rate.max() if len(heart_rate) > 0 else 0
        std_hr = heart_rate.std() if len(heart_rate) > 0 else 0
        high_hr = heart_rate[heart_rate > 100].sum()

        return mean_hr, min_hr, max_hr, std_hr, high_hr

    # 하루
    active_hour_mean_hr, active_hour_min_hr, active_hour_max_hr, active_hour_std_hr, active_hour_high_hr = _process_feature(heart_rate[df["hour"].isin(ACTIVE_HOURS)])

    # 잠자는 시간대
    sleep_hour_mean_hr, sleep_hour_min_hr, sleep_hour_max_hr, sleep_hour_std_hr, sleep_hour_high_hr = _process_feature(heart_rate[df["hour"].isin(SLEEP_HOURS)])

    return pd.Series({
        'active_hour_mean_hr': active_hour_mean_hr,
        'active_hour_min_hr': active_hour_min_hr,
        'active_hour_max_hr': active_hour_max_hr,
        'active_hour_std_hr': active_hour_std_hr,
        'active_hour_high_hr': active_hour_high_hr,
        'sleep_hour_mean_hr': sleep_hour_mean_hr,
        'sleep_hour_min_hr': sleep_hour_min_hr,
        'sleep_hour_max_hr': sleep_hour_max_hr,
        'sleep_hour_std_hr': sleep_hour_std_hr,
        'sleep_hour_high_hr': sleep_hour_high_hr
    })

wHr_ori = load_data(DataType.wHr)
wHr_ori = shift_lifelog_date(wHr_ori, target_hours=SLEEP_HOURS)

wHr2 = (
    wHr_ori
    .groupby(["subject_id", "lifelog_date"], group_keys=False, as_index=False, sort=False, observed=True)
    .apply(process_wHr)
    .reset_index(drop=True)
)

describe_df(wHr2)

# shape:
(679, 12)

# dtypes:
subject_id                   category
lifelog_date           datetime64[ns]
active_hour_mean_hr           float64
active_hour_min_hr            float64
active_hour_max_hr            float64
active_hour_std_hr            float64
active_hour_high_hr           float64
sleep_hour_mean_hr            float64
sleep_hour_min_hr             float64
sleep_hour_max_hr             float64
sleep_hour_std_hr             float64
sleep_hour_high_hr            float64
dtype: object



,subject_id,lifelog_date,active_hour_mean_hr,active_hour_min_hr,active_hour_max_hr,active_hour_std_hr,active_hour_high_hr,sleep_hour_mean_hr,sleep_hour_min_hr,sleep_hour_max_hr,sleep_hour_std_hr,sleep_hour_high_hr
0,id01,2024-06-26,81.2434,59.0000,142.0000,11.8712,243191.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,id01,2024-06-27,79.3523,53.0000,130.0000,12.6371,119052.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,id01,2024-06-28,77.3601,51.0000,135.0000,12.5109,142587.0000,0.0000,0.0000,0.0000,0.0000,0.0000


# nan_stats:
                     missing_count  missing_ratio(%)
subject_id                       0            0.0000
lifelog_date                     0            0.0000
active_hour_mean_hr              0            0.0000
active_hour_min_hr               0            0.0000
active_hour_max_hr               0            0.0000
active_hour_std_hr               0            0.0000
active_hour_high_hr              0            0.0000
sleep_hour_mean_hr               0            0.0000
sleep_hour_min_hr                0            0.0000
sleep_hour_max_hr                0            0.0000
sleep_hour_std_hr                0            0.0000
sleep_hour_high_hr               0            0.0000



### ✔️ wLight 앰비언트 라이트
- Ambient light measured by the smartwatch.  
  - 어두운 밤 0.1 ~ 1 lux 캄캄한 방, 달빛 없는 밤
  - 가로등 켜진 거리 10 ~ 20 lux 흐릿한 외부 조명
  - 실내 조명 100 ~ 500 lux 사무실, 일반 거실
  - 밝은 실외 10,000 ~ 25,000 lux 맑은 날 햇빛
  - 직사광선 아래 30,000 ~ 100,000 lux 여름 한낮, 매우 강한 햇빛

In [40]:
def process_mLight(df):
    light = df["m_light"].values  # [534.0, 224, ...]

    def _process_feature(light):
        if len(light) == 0:
            return 0., 0., 0., 0., np.array([])

        ligths = np.array(light)
        mean_light = ligths.mean() if len(ligths) > 0 else 0
        min_light = ligths.min() if len(ligths) > 0 else 0
        max_light = ligths.max() if len(ligths) > 0 else 0
        std_light = ligths.std() if len(ligths) > 0 else 0

        return mean_light, min_light, max_light, std_light, ligths

    # 하루
    active_hour_mean_light, active_hour_min_light, active_hour_max_light, active_hour_std_light, _ = _process_feature(light[df["hour"].isin(ACTIVE_HOURS)])

    # 잠자는 시간대
    sleep_hour_mean_light, sleep_hour_min_light, sleep_hour_max_light, sleep_hour_std_light, _= _process_feature(light[df["hour"].isin(SLEEP_HOURS)])

    # 잠 자러갈 때
    might_go_to_sleep_light = light[df["hour"].isin(MIGHT_GO_TO_SLEEP_HOURS)]
    might_go_to_sleep_timestamps = df["timestamp"].values[df["hour"].isin(MIGHT_GO_TO_SLEEP_HOURS)]
    _, _, _, _, might_go_to_sleep_lights = _process_feature(might_go_to_sleep_light)
    first_sleep_timestamps = might_go_to_sleep_timestamps[(might_go_to_sleep_lights < 10.0)]
    first_sleep_datetime = (
        pd.to_datetime(first_sleep_timestamps[0]) if len(first_sleep_timestamps) > 0
        else pd.to_datetime(might_go_to_sleep_timestamps[-1]) if len(might_go_to_sleep_timestamps) > 0
        else pd.to_datetime(datetime(2024, 1, 1, MIGHT_GO_TO_SLEEP_HOURS[-1], 0, 0))  # default to the last hour of the range
    )
    first_sleep_minutes = (first_sleep_datetime.hour * 60 if first_sleep_datetime.hour > 12 else (first_sleep_datetime.hour + 24) * 60) + first_sleep_datetime.minute

    # 일어날 때
    might_wakeup_light = light[df["hour"].isin(MIGHT_WAKEUP_HOURS)]
    might_wakeup_timestamps = df["timestamp"].values[df["hour"].isin(MIGHT_WAKEUP_HOURS)]
    _, _, _, _, might_wakeup_lights = _process_feature(might_wakeup_light)
    wakeup_timestamps = might_wakeup_timestamps[(might_wakeup_lights > 10.0)]
    first_move_datetime = (
        pd.to_datetime(wakeup_timestamps[0]) if len(wakeup_timestamps) > 0
        else pd.to_datetime(might_wakeup_timestamps[-1]) if len(might_wakeup_timestamps) > 0
        else pd.to_datetime(datetime(2024, 1, 1, MIGHT_WAKEUP_HOURS[-1], 0, 0))  # default to the last hour of the range
    )
    first_wakeup_minutes = (first_move_datetime.hour if first_move_datetime.hour > 12 else first_move_datetime.hour + 24) * 60 + first_move_datetime.minute

    return pd.Series({
        'active_hour_mean_light': active_hour_mean_light,
        'active_hour_min_light': active_hour_min_light,
        'active_hour_max_light': active_hour_max_light,
        'active_hour_std_light': active_hour_std_light,
        'sleep_hour_mean_light': sleep_hour_mean_light,
        'sleep_hour_min_light': sleep_hour_min_light,
        'sleep_hour_max_light': sleep_hour_max_light,
        'sleep_hour_std_light': sleep_hour_std_light,
        'mlight_first_sleep_minutes': first_sleep_minutes,
        'mlight_first_wakeup_minutes': first_wakeup_minutes,
    })

wLight_ori = load_data(DataType.wLight)
wLight_ori = shift_lifelog_date(wLight_ori, target_hours=SLEEP_HOURS)

wLight2 = (
    wLight_ori
    .rename(columns={"w_light": "m_light"})
    .groupby(["subject_id", "lifelog_date"], group_keys=False, as_index=False, sort=False, observed=True)
    .apply(process_mLight)
    .reset_index(drop=True)
)

wLight2.rename(
    columns={
        col: "w" + col.replace("mlight_", "wlight_")
        for col in wLight.columns if col not in ["subject_id", "lifelog_date"]
    }, inplace=True
)

describe_df(wLight2)

# shape:
(752, 12)

# dtypes:
subject_id                           category
lifelog_date                   datetime64[ns]
active_hour_mean_light                float64
active_hour_min_light                 float64
active_hour_max_light                 float64
active_hour_std_light                 float64
sleep_hour_mean_light                 float64
sleep_hour_min_light                  float64
sleep_hour_max_light                  float64
sleep_hour_std_light                  float64
mlight_first_sleep_minutes            float64
mlight_first_wakeup_minutes           float64
dtype: object



,subject_id,lifelog_date,active_hour_mean_light,active_hour_min_light,active_hour_max_light,active_hour_std_light,sleep_hour_mean_light,sleep_hour_min_light,sleep_hour_max_light,sleep_hour_std_light,mlight_first_sleep_minutes,mlight_first_wakeup_minutes
0,id01,2024-06-26,299.4151,0.0000,20874.0000,1220.1126,0.0000,0.0000,0.0000,0.0000,1274.0000,1980.0000
1,id01,2024-06-27,290.7522,0.0000,12464.0000,1031.2252,0.0000,0.0000,0.0000,0.0000,1200.0000,1830.0000
2,id01,2024-06-28,518.8263,0.0000,91584.0000,3794.5034,0.0000,0.0000,0.0000,0.0000,1209.0000,1835.0000


# nan_stats:
                             missing_count  missing_ratio(%)
subject_id                               0            0.0000
lifelog_date                             0            0.0000
active_hour_mean_light                   0            0.0000
active_hour_min_light                    0            0.0000
active_hour_max_light                    0            0.0000
active_hour_std_light                    0            0.0000
sleep_hour_mean_light                    0            0.0000
sleep_hour_min_light                     0            0.0000
sleep_hour_max_light                     0            0.0000
sleep_hour_std_light                     0            0.0000
mlight_first_sleep_minutes               0            0.0000
mlight_first_wakeup_minutes              0            0.0000



### ✔️ wPedo 걸음수
- Step data recorded by the smartwatch.

In [41]:
def process_mPedo(df):

    def _process_feature(df):
        if len(df) == 0:
            return 0., 0., 0.

        steps = df["step"].values
        distances = df["distance"].values
        calories = df["burned_calories"].values

        steps = steps.sum() if len(steps) > 0 else 0
        distance = distances.sum() if len(distances) > 0 else 0
        burned_calories = calories.sum() if len(calories) > 0 else 0

        return steps, distance, burned_calories

    # 하루
    active_hour_steps, active_hour_distance, active_hour_burned_calories = _process_feature(df[df["hour"].isin(ACTIVE_HOURS)])

    # 잠자는 시간대
    sleep_hour_steps, sleep_hour_distance, sleep_hour_burned_calories = _process_feature(df[df["hour"].isin(SLEEP_HOURS)])

    return pd.Series({
        'active_hour_steps': active_hour_steps,
        'active_hour_distance': active_hour_distance,
        'active_hour_burned_calories': active_hour_burned_calories,
        'sleep_hour_steps': sleep_hour_steps,
        'sleep_hour_distance': sleep_hour_distance,
        'sleep_hour_burned_calories': sleep_hour_burned_calories
    })

wPedo_ori = load_data(DataType.wPedo)
wPedo_ori = shift_lifelog_date(wPedo_ori, target_hours=SLEEP_HOURS)

wPedo2 = (
    wPedo_ori
    .groupby(["subject_id", "lifelog_date"], group_keys=False, as_index=False, sort=False, observed=True)
    .apply(process_mPedo)
    .reset_index(drop=True)
)

describe_df(wPedo2)

# shape:
(735, 8)

# dtypes:
subject_id                           category
lifelog_date                   datetime64[ns]
active_hour_steps                     float64
active_hour_distance                  float64
active_hour_burned_calories           float64
sleep_hour_steps                      float64
sleep_hour_distance                   float64
sleep_hour_burned_calories            float64
dtype: object



,subject_id,lifelog_date,active_hour_steps,active_hour_distance,active_hour_burned_calories,sleep_hour_steps,sleep_hour_distance,sleep_hour_burned_calories
0,id01,2024-06-26,3578.0000,2782.1901,189.3191,0.0000,0.0000,0.0000
1,id01,2024-06-27,2619.0000,2020.5527,280.2708,10.0000,6.8300,0.0000
2,id01,2024-06-28,3726.0000,2888.0892,116.1595,0.0000,0.0000,0.0000


# nan_stats:
                             missing_count  missing_ratio(%)
subject_id                               0            0.0000
lifelog_date                             0            0.0000
active_hour_steps                        0            0.0000
active_hour_distance                     0            0.0000
active_hour_burned_calories              0            0.0000
sleep_hour_steps                         0            0.0000
sleep_hour_distance                      0            0.0000
sleep_hour_burned_calories               0            0.0000



### ✔️ 잠에서 깨어난 건수

In [42]:
def compute_night_awake_features(df, prefix):

    df = df.copy()
    df['lifelog_date'] = df['timestamp'].astype(str).str[:10]
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # 00시~06시 필터
    df['hour'] = df['timestamp'].dt.hour
    df_night = df[(df['hour'] >= 0) & (df['hour'] < 6)].copy()

    # 깨어있는 분 계산
    df_night['awake_minute'] = (df_night[prefix] > 0).astype(int)

    # 깨어난 횟수 계산 (0 → 양수 전환)
    def count_awake_blocks(x):
        return ((x > 0) & (x.shift(fill_value=0) == 0)).sum()

    # 그룹별 집계
    result = df_night.groupby(['subject_id', 'lifelog_date']).agg(
        awake_minutes=('awake_minute', 'sum'),
        awake_blocks=(prefix, count_awake_blocks)
    ).reset_index()

    # 컬럼명 변경
    result = result.rename(columns={
        'awake_minutes': f'{prefix}_awake_minutes',
        'awake_blocks': f'{prefix}_awake_blocks'
    })

    # train에 결과 합치기 위해서 -1 day 하기
    result['lifelog_date'] = pd.to_datetime(result['lifelog_date'])
    result['lifelog_date'] = result['lifelog_date'] + pd.Timedelta(days=-1)
    result['lifelog_date'] = result['lifelog_date'].astype(str)

    return result

In [43]:
mAwakeBlocks = compute_night_awake_features(mLight,'m_light')
wAwakeBlocks = compute_night_awake_features(wLight,'w_light')
AwakeBlocks = mAwakeBlocks.merge(wAwakeBlocks, on=['subject_id','lifelog_date'], how='outer')
AwakeBlocks['light_awake_minutes'] = AwakeBlocks[['m_light_awake_minutes','w_light_awake_minutes']].max(axis=1)
AwakeBlocks['light_awake_blocks'] = AwakeBlocks[['m_light_awake_blocks','w_light_awake_blocks']].max(axis=1)

# check
print(mAwakeBlocks.shape)
print(wAwakeBlocks.shape)
print(AwakeBlocks.shape)

(677, 4)
(565, 4)
(687, 8)


### 📌 merge 데이터
- train, test 기간 서로 겹침

In [44]:
train = pd.read_csv(f'{PATH}/ETRI_lifelog_dataset/ch2025_metrics_train.csv')
test = pd.read_csv(f'{PATH}/ETRI_lifelog_dataset/ch2025_submission_sample.csv')

# 일자변수 타입 변환
mACStatus2['lifelog_date'] = mACStatus2['lifelog_date'].astype(str)
mActivity21['lifelog_date'] = mActivity21['lifelog_date'].astype(str)
mActivity22['lifelog_date'] = mActivity22['lifelog_date'].astype(str)
mAmbience2['lifelog_date'] = mAmbience2['lifelog_date'].astype(str)
mBle2['lifelog_date'] = mBle2['lifelog_date'].astype(str)
mGps2['lifelog_date'] = mGps2['lifelog_date'].astype(str)
mLight21['lifelog_date'] = mLight21['lifelog_date'].astype(str)
mLight22['lifelog_date'] = mLight22['lifelog_date'].astype(str)
mLight23['lifelog_date'] = mLight23['lifelog_date'].astype(str)
mScreenStatus2['lifelog_date'] = mScreenStatus2['lifelog_date'].astype(str)
mUsageStats2['lifelog_date'] = mUsageStats2['lifelog_date'].astype(str)
mWifi2['lifelog_date'] = mWifi2['lifelog_date'].astype(str)
wHr2['lifelog_date'] = wHr2['lifelog_date'].astype(str)
wLight2['lifelog_date'] = wLight2['lifelog_date'].astype(str)
wPedo2['lifelog_date'] = wPedo2['lifelog_date'].astype(str)

# ---- new ----

AwakeBlocks['lifelog_date'] = AwakeBlocks['lifelog_date'].astype(str)

In [45]:
df_list = [
    mACStatus2,       # 1
    mActivity21,       # 2
    mActivity22,       # 2
    mAmbience2,       # 3
    mBle2,            # 4
    mGps2,            # 5
    mLight21,          # 6
    mLight22,          # 6
    mLight23,          # 6
    mScreenStatus2,   # 7
    mUsageStats2,     # 8
    mWifi2,           # 9
    wHr2,             # 10
    wLight2,          # 11
    wPedo2,           # 12
    # ---- new ----
    AwakeBlocks
]

data = reduce(lambda left, right: pd.merge(left, right, on=['subject_id', 'lifelog_date'], how='outer'), df_list)
data['lifelog_date'] = data['lifelog_date'].astype(str)

# 중복체크
print(data.shape)
print(data[['subject_id','lifelog_date']].drop_duplicates().shape)

# merge
train2 = train.merge(data, on=['subject_id','lifelog_date'], how='left')
test2 = test.merge(data, on=['subject_id','lifelog_date'], how='left')

# 저장
print('# train  shape:',train.shape)
print('# train2 shape:',test2.shape)
print('# test   shape:',test.shape)
print('# test2  shape:',test2.shape)

(806, 245)
(806, 2)
# train  shape: (450, 9)
# train2 shape: (250, 252)
# test   shape: (250, 9)
# test2  shape: (250, 252)


### 📁 데이터 저장

In [46]:
train = train2.copy()
test = test2.copy()

# drop_features = ['afterwork_max_label','sleeptime_max_label','worktime_max_label']
drop_features = ['top_bssid'] # ,'week_type','week_type_lag1'
drop_features = [i for i in drop_features if i in train.columns.tolist()]
print('# drop_features:',drop_features)
train = train.drop(columns=drop_features)
test = test.drop(columns=drop_features)

# drop_features: []


In [47]:
train['불끈시간부터기상시간'] = train.apply(lambda x: calculate_sleep_duration_min(x['lights_off_time'],x['wake_time']),axis=1)
test['불끈시간부터기상시간'] = test.apply(lambda x: calculate_sleep_duration_min(x['lights_off_time'],x['wake_time']),axis=1)

In [48]:
train['추정수면효율'] = train['불끈시간부터기상시간']/train['sleep_duration_min']
test['추정수면효율'] = test['불끈시간부터기상시간']/test['sleep_duration_min']

# 이상값 제거
train['추정수면효율'] = np.where(train['추정수면효율']<-5,np.nan,train['추정수면효율'])
test['추정수면효율'] = np.where(test['추정수면효율']<-5,np.nan,test['추정수면효율'])
train['추정수면효율'] = np.where(train['추정수면효율']>5,np.nan,train['추정수면효율'])
test['추정수면효율'] = np.where(test['추정수면효율']>55,np.nan,test['추정수면효율'])

In [49]:
# 요일 컬럼 추가 (예: 월요일, 화요일, ...)
train['lifelog_date'] = pd.to_datetime(train['lifelog_date'])
test['lifelog_date'] = pd.to_datetime(test['lifelog_date'])

# 요일
weekday_map = {
    0: '월요일', 1: '화요일', 2: '수요일', 3: '목요일',
    4: '금요일', 5: '토요일', 6: '일요일'
}
train['weekday'] = train['lifelog_date'].dt.dayofweek.map(weekday_map)
test['weekday'] = test['lifelog_date'].dt.dayofweek.map(weekday_map)

# 월
train['month'] = train['lifelog_date'].dt.month
test['month'] = test['lifelog_date'].dt.month

# weekend
train['weekend'] = np.where(train['weekday'].isin(['토요일','일요일']),1,0)
test['weekend'] = np.where(test['weekday'].isin(['토요일','일요일']),1,0)

# 공휴일
공휴일 = [
     '2024-08-15'
    ,'2024-09-16'
    ,'2024-09-17'
    ,'2024-09-18'
    ,'2024-10-03'
    ,'2024-10-09'
]
train['공휴일'] = np.where(train['lifelog_date'].isin(공휴일),1,0)
test['공휴일'] = np.where(test['lifelog_date'].isin(공휴일),1,0)

# 주말 + 공휴일 묶어주기
train['weekend_holilday'] = np.where( ((train['weekend']==0) & (train['공휴일']==1)), 1, train['weekend'])
test['weekend_holilday'] = np.where( ((test['weekend']==0) & (test['공휴일']==1)), 1, test['weekend'])

In [50]:
def add_prev_day_flag(df):
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])

    # 각 subject_id별로 전날 날짜 만들기
    df['prev_day'] = df['lifelog_date'] - pd.Timedelta(days=1)

    # subject_id + 날짜 기준으로 원본 키 구성
    key_set = set(zip(df['subject_id'], df['lifelog_date']))

    # 전날 데이터가 존재하면 1, 없으면 0
    df['has_prev_day_data'] = df[['subject_id', 'prev_day']].apply(
        lambda row: 1 if (row['subject_id'], row['prev_day']) in key_set else 0, axis=1
    )

    return df.drop(columns=['prev_day'])

train = add_prev_day_flag(train)
test = add_prev_day_flag(test)

In [51]:
def rule_based_sum(x):
    rules = (
        # (x['sleep_duration_min'] > (x['avg_sleep_duration']+30))
          (x['sleep_duration_min'] > (x['avg_sleep_duration']+60))
        & (x['week_type'] == 'weekday')
        # & (x['month'].isin([7,8]))
    )
    return rules

# 추정휴가
train['vacation'] = train.groupby('subject_id').apply(rule_based_sum).reset_index(level=0, drop=True).astype(int)
test['vacation'] = test.groupby('subject_id').apply(rule_based_sum).reset_index(level=0, drop=True).astype(int)

# check
test.groupby(['subject_id'])['vacation'].sum().head()

,vacation
subject_id,
id01,3
id02,3
id03,3
id04,2
id05,1


In [52]:
# 저장
train.to_parquet(f"{PATH}/train_63775_vF.parquet")
test.to_parquet(f"{PATH}/test_63775_vF.parquet")